In [49]:
import boto3
import pandas as pd
import numpy as np
import os
import math
import datetime
from collections import deque
from scipy.stats import norm

# ==========================================
# 1. CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
DAY = 20
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"  # Trading symbol of futures on that date

# Strategy Parameters
LOT_SIZE = 75
RISK_FREE_RATE = 0.0          

# COST PARAMETERS (REALISTIC)
# We only pay this ONCE per trade now (Single Leg), not twice (Straddle)
OPTION_ENTRY_SPREAD = 2.0    
OPTION_EXIT_SPREAD = 2.0  
OPTION_ENTRY_SLIPPAGE = 1.0  
OPTION_EXIT_SLIPPAGE = 1.0

# Total costs per leg
ENTRY_COST_PER_LEG = OPTION_ENTRY_SPREAD + OPTION_ENTRY_SLIPPAGE  # 3 pts
EXIT_COST_PER_LEG = OPTION_EXIT_SPREAD + OPTION_EXIT_SLIPPAGE     # 3 pts

BASE_FOLDER = f"data/{YEAR}-{MONTH:02d}-{DAY:02d}/"
os.makedirs(BASE_FOLDER, exist_ok=True)

print(f"--- RUNNING DIRECTIONAL CALL SCALPER FOR {DAY}-{MONTH}-{YEAR} ---")

# ==========================================
# 2. S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation: kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]: keys.append(obj["Key"])
        if resp.get("IsTruncated"): continuation = resp.get("NextContinuationToken")
        else: break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return # Skip if already exists
    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=BUCKET, Key=key)
    with open(local_path, "wb") as f:
        f.write(obj["Body"].read())

# ==========================================
# 3. FUTURES & KINETIC LOGIC
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(BASE_FOLDER, "FUT.parquet")
    print(f"1. Downloading Futures: {key}")
    download_parquet_to_path(key, local_path)
    
    # Convert to CSV for easier processing logic
    df = pd.read_parquet(local_path)
    df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str), errors="coerce")
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Handle Volume vs OpenInterest naming variations
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    csv_path = os.path.join(BASE_FOLDER, "FUT.csv")
    df.to_csv(csv_path, index=False)
    return csv_path

class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=1200):
        self.buf = deque(maxlen=50)
        self.in_trade = False
        self.entry_time = None
        self.threshold = threshold
        self.hold_seconds = hold_seconds

    def tick(self, ts, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0

        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1 # EXIT
            return 0 # HOLD

        arr = np.array(self.buf)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        disp = abs(prices[-1] - prices[0])
        score = traded_vol / (disp + 0.05)

        if score > self.threshold:
            self.in_trade = True
            self.entry_time = ts
            return 1 # ENTER

        return 0

def build_kinetic_trades(fut_csv):
    print("2. Generating Kinetic Signals...")
    df = pd.read_csv(fut_csv)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    
    brain = KineticBrain()
    trades = []
    active = None

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        sig = brain.tick(ts, ltp, vol)

        if sig == 1:
            active = {
                "Date": ts.date(),
                "Entry_Time": ts,
                "Entry_LTP": ltp,
                # FIX: ATM Strike determined at ENTRY, not Average
                "ATM_Strike": round(ltp / 50.0) * 50
            }
        elif sig == -1 and active is not None:
            active["Exit_Time"] = ts
            active["Exit_LTP"] = ltp
            trades.append(active)
            active = None

    trades_df = pd.DataFrame(trades)
    out_path = os.path.join(BASE_FOLDER, "kinetic_trades.csv")
    trades_df.to_csv(out_path, index=False)
    print(f"   Generated {len(trades_df)} trades.")
    return out_path

# ==========================================
# 4. EXPIRY & OPTION DOWNLOAD LOGIC
# ==========================================
month_map = {"JAN":1,"FEB":2,"MAR":3,"APR":4,"MAY":5,"JUN":6,"JUL":7,"AUG":8,"SEP":9,"OCT":10,"NOV":11,"DEC":12}

def get_expiry_date_obj(code, ref_date):
    """Converts code like '25NOV' to datetime object"""
    y = 2000 + int(code[:2])
    tail = code[2:]
    if tail in month_map:
        return datetime.datetime(y, month_map[tail], 28) 
    return datetime.datetime(2025, 11, 27) 

def find_nearest_expiry(year, month, day, symbol):
    # Simplified S3 lookup
    base_prefix = f"year={year}/month={month:02d}/day={day:02d}/Options/{symbol}/"
    keys = list_s3(base_prefix)
    expiries = set()
    for k in keys:
        try:
            # key: .../Options/NIFTY/25NOV/CE...
            suffix = k.split(base_prefix, 1)[1]
            part = suffix.split("/", 1)[0]
            expiries.add(part)
        except: continue
    
    if not expiries: raise Exception("No Expiries Found")
    return sorted(list(expiries))[0]

def download_specific_options(year, month, day, symbol, expiry_code, strikes):
    print(f"3. Downloading Options for Strikes: {strikes}")
    local_map = {} # (Strike, Type) -> Path
    
    base_prefix = f"year={year}/month={month:02d}/day={day:02d}/Options/{symbol}/{expiry_code}/"
    
    for strike in strikes:
        # STRATEGY CHANGE: We only download CE (Call) for Directional Long
        for opt_type in ["CE"]:
            key = f"{base_prefix}{opt_type}/{strike}/{symbol}{expiry_code}{strike}{opt_type}.parquet"
            local_name = f"{symbol}{expiry_code}{strike}{opt_type}.parquet"
            local_path = os.path.join(BASE_FOLDER, local_name)
            
            try:
                download_parquet_to_path(key, local_path)
                local_map[(strike, opt_type)] = local_path
            except Exception:
                print(f"   MISSING: {key}")
                
    return local_map

# ==========================================
# 5. DIRECTIONAL SCALPER ENGINE
# ==========================================
def run_directional_engine(trades_csv, option_map):
    print("\n4. Running Directional Call Simulation...")
    
    trades_df = pd.read_csv(trades_csv)
    trades_df['Entry_Time'] = pd.to_datetime(trades_df['Entry_Time'])
    trades_df['Exit_Time'] = pd.to_datetime(trades_df['Exit_Time'])
    
    results = []
    
    for i, row in trades_df.iterrows():
        entry_ts = row['Entry_Time']
        exit_ts = row['Exit_Time']
        K = int(row['ATM_Strike'])
        
        # We only need the CE file
        ce_path = option_map.get((K, 'CE'))
        if not ce_path:
            print(f"   Skip Trade {i}: Missing CE file for Strike {K}")
            continue
            
        try:
            ce_df = pd.read_parquet(ce_path)
            if "DateTime" not in ce_df.columns:
                 ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))
            ce_df.set_index("DateTime", inplace=True)
            if "LastTradedPrice" in ce_df.columns: ce_df["LTP"] = ce_df["LastTradedPrice"]
            ce_df = ce_df.sort_index()
        except Exception as e:
            print(f"   Error reading parquet: {e}")
            continue

        # Get Precise Entry/Exit Prices
        # We use 'searchsorted' logic implicitly via reindex or nearest lookup
        # But slicing is safer to see if data exists
        
        trade_data = ce_df.loc[entry_ts:exit_ts]
        if trade_data.empty:
            continue
            
        # ENTRY LOGIC
        # We assume we enter exactly at signal time (or next available tick)
        # Realistically: Ask Price + Slippage
        # Approximation: LTP + Entry Cost
        entry_price_raw = trade_data.iloc[0]['LTP']
        entry_price_net = entry_price_raw + ENTRY_COST_PER_LEG
        
        # EXIT LOGIC
        # We exit at signal time
        # Realistically: Bid Price - Slippage
        # Approximation: LTP - Exit Cost
        exit_price_raw = trade_data.iloc[-1]['LTP']
        exit_price_net = exit_price_raw - EXIT_COST_PER_LEG
        
        # PnL Calculation
        pnl_points = exit_price_net - entry_price_net
        
        results.append({
            "Entry_Time": entry_ts,
            "Exit_Time": exit_ts,
            "Strike": K,
            "CE_Entry_Raw": entry_price_raw,
            "CE_Exit_Raw": exit_price_raw,
            "PnL_Points": pnl_points,
            "PnL_INR": pnl_points * LOT_SIZE
        })

    # Save Results
    res_df = pd.DataFrame(results)
    final_csv = os.path.join(BASE_FOLDER, "directional_scalping_results.csv")
    res_df.to_csv(final_csv, index=False)
    
    print("\n--- RESULTS ---")
    print(f"Total Trades: {len(res_df)}")
    if not res_df.empty:
        print(f"Total Net PnL: ₹{res_df['PnL_INR'].sum():,.2f}")
        print(f"Win Rate: {(res_df['PnL_Points'] > 0).mean()*100:.1f}%")
        print(f"Avg PnL per Trade: ₹{res_df['PnL_INR'].mean():,.2f}")
    else:
        print("No valid trades found.")
    print(f"Saved to: {final_csv}")

# ==========================================
# 6. MAIN EXECUTION
# ==========================================
def main():
    # A. Get Futures
    fut_csv = download_futures_for_day(YEAR, MONTH, DAY, SYMBOL, FUT_TS)
    
    # B. Generate Signals
    trades_csv = build_kinetic_trades(fut_csv)
    
    # C. Identify Needed Options
    trades_df = pd.read_csv(trades_csv)
    if trades_df.empty:
        print("No trades generated.")
        return
        
    needed_strikes = trades_df["ATM_Strike"].unique().astype(int).tolist()
    expiry_code = find_nearest_expiry(YEAR, MONTH, DAY, SYMBOL)
    print(f"Target Expiry: {expiry_code}")
    
    # D. Get Options (ONLY CE)
    opt_map = download_specific_options(YEAR, MONTH, DAY, SYMBOL, expiry_code, needed_strikes)
    
    # E. Run Engine
    run_directional_engine(trades_csv, opt_map)

if __name__ == "__main__":
    main()

--- RUNNING DIRECTIONAL CALL SCALPER FOR 20-11-2025 ---
1. Downloading Futures: year=2025/month=11/day=20/Futures/NIFTY/NIFTY25NOVFUT.parquet


/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/696557002.py:76: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str), errors="coerce")


2. Generating Kinetic Signals...
   Generated 18 trades.
Target Expiry: 25D02
3. Downloading Options for Strikes: [26150, 26100, 26200, 26250]

4. Running Directional Call Simulation...


/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/696557002.py:233: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))
/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/696557002.py:233: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))


KeyboardInterrupt: 

In [ ]:
import boto3
import pandas as pd
import numpy as np
import os
import math
import datetime
from collections import deque
from scipy.stats import norm

# ==========================================
# 1. CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
DAY = 20
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"  # Trading symbol of futures on that date

# Strategy Parameters
LOT_SIZE = 75
RISK_FREE_RATE = 0.0          

# COST PARAMETERS (REALISTIC)
# We only pay this ONCE per trade now (Single Leg), not twice (Straddle)
OPTION_ENTRY_SPREAD = 2.0    
OPTION_EXIT_SPREAD = 2.0  
OPTION_ENTRY_SLIPPAGE = 1.0  
OPTION_EXIT_SLIPPAGE = 1.0
FUTURES_SLIPPAGE = 0.5       # Per contract per side (Total 1.0 round trip)

# Total costs per leg
ENTRY_COST_PER_LEG = OPTION_ENTRY_SPREAD + OPTION_ENTRY_SLIPPAGE  # 3 pts
EXIT_COST_PER_LEG = OPTION_EXIT_SPREAD + OPTION_EXIT_SLIPPAGE     # 3 pts

BASE_FOLDER = f"data/{YEAR}-{MONTH:02d}-{DAY:02d}/"
os.makedirs(BASE_FOLDER, exist_ok=True)

print(f"--- RUNNING DEEP ITM CALL SCALPER (DELTA ~0.85) FOR {DAY}-{MONTH}-{YEAR} ---")

# ==========================================
# 2. S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation: kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]: keys.append(obj["Key"])
        if resp.get("IsTruncated"): continuation = resp.get("NextContinuationToken")
        else: break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return # Skip if already exists
    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=BUCKET, Key=key)
    with open(local_path, "wb") as f:
        f.write(obj["Body"].read())

# ==========================================
# 3. FUTURES & KINETIC LOGIC
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(BASE_FOLDER, "FUT.parquet")
    print(f"1. Downloading Futures: {key}")
    download_parquet_to_path(key, local_path)
    
    # Convert to CSV for easier processing logic
    df = pd.read_parquet(local_path)
    df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str), errors="coerce")
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Handle Volume vs OpenInterest naming variations
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    csv_path = os.path.join(BASE_FOLDER, "FUT.csv")
    df.to_csv(csv_path, index=False)
    return csv_path

class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=1200):
        self.buf = deque(maxlen=50)
        self.in_trade = False
        self.entry_time = None
        self.threshold = threshold
        self.hold_seconds = hold_seconds

    def tick(self, ts, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0

        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1 # EXIT
            return 0 # HOLD

        arr = np.array(self.buf)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        disp = abs(prices[-1] - prices[0])
        score = traded_vol / (disp + 0.05)

        if score > self.threshold:
            self.in_trade = True
            self.entry_time = ts
            return 1 # ENTER

        return 0

def build_kinetic_trades(fut_csv):
    print("2. Generating Kinetic Signals...")
    df = pd.read_csv(fut_csv)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    
    brain = KineticBrain()
    trades = []
    active = None

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        sig = brain.tick(ts, ltp, vol)

        if sig == 1:
            active = {
                "Date": ts.date(),
                "Entry_Time": ts,
                "Entry_LTP": ltp,
                # FIX: SELECT DEEP ITM STRIKE (LTP - 200)
                # This gives us Delta ~0.85 to capture more of the move
                "ATM_Strike": (round(ltp / 50.0) * 50) - 200
            }
        elif sig == -1 and active is not None:
            active["Exit_Time"] = ts
            active["Exit_LTP"] = ltp
            trades.append(active)
            active = None

    trades_df = pd.DataFrame(trades)
    out_path = os.path.join(BASE_FOLDER, "kinetic_trades.csv")
    trades_df.to_csv(out_path, index=False)
    print(f"   Generated {len(trades_df)} trades.")
    return out_path

# ==========================================
# 4. EXPIRY & OPTION DOWNLOAD LOGIC
# ==========================================
month_map = {"JAN":1,"FEB":2,"MAR":3,"APR":4,"MAY":5,"JUN":6,"JUL":7,"AUG":8,"SEP":9,"OCT":10,"NOV":11,"DEC":12}

def get_expiry_date_obj(code, ref_date):
    """Converts code like '25NOV' to datetime object"""
    y = 2000 + int(code[:2])
    tail = code[2:]
    if tail in month_map:
        return datetime.datetime(y, month_map[tail], 28) 
    return datetime.datetime(2025, 11, 27) 

def find_nearest_expiry(year, month, day, symbol):
    # Simplified S3 lookup
    base_prefix = f"year={year}/month={month:02d}/day={day:02d}/Options/{symbol}/"
    keys = list_s3(base_prefix)
    expiries = set()
    for k in keys:
        try:
            # key: .../Options/NIFTY/25NOV/CE...
            suffix = k.split(base_prefix, 1)[1]
            part = suffix.split("/", 1)[0]
            expiries.add(part)
        except: continue
    
    if not expiries: raise Exception("No Expiries Found")
    return sorted(list(expiries))[0]

def download_specific_options(year, month, day, symbol, expiry_code, strikes):
    print(f"3. Downloading Options for Strikes: {strikes}")
    local_map = {} # (Strike, Type) -> Path
    
    base_prefix = f"year={year}/month={month:02d}/day={day:02d}/Options/{symbol}/{expiry_code}/"
    
    for strike in strikes:
        # STRATEGY CHANGE: We only download CE (Call) for Directional Long
        for opt_type in ["CE"]:
            key = f"{base_prefix}{opt_type}/{strike}/{symbol}{expiry_code}{strike}{opt_type}.parquet"
            local_name = f"{symbol}{expiry_code}{strike}{opt_type}.parquet"
            local_path = os.path.join(BASE_FOLDER, local_name)
            
            try:
                download_parquet_to_path(key, local_path)
                local_map[(strike, opt_type)] = local_path
            except Exception:
                print(f"   MISSING: {key}")
                
    return local_map

# ==========================================
# 5. CONTROL GROUP: FUTURES BENCHMARK
# ==========================================
def run_futures_benchmark(trades_csv):
    print("\n--- CONTROL GROUP: RAW FUTURES PERFORMANCE ---")
    df = pd.read_csv(trades_csv)
    if df.empty:
        print("No trades found.")
        return

    # Raw Move: Exit - Entry
    df['Raw_Pts'] = df['Exit_LTP'] - df['Entry_LTP']
    
    # Futures Costs: Entry Slippage + Exit Slippage (1.0 Total)
    # Brokerage is negligible compared to this, but let's assume 0.5 total extra
    TOTAL_FUT_COST = FUTURES_SLIPPAGE + FUTURES_SLIPPAGE 
    
    df['Net_Pts'] = df['Raw_Pts'] - TOTAL_FUT_COST
    df['Net_INR'] = df['Net_Pts'] * LOT_SIZE
    
    total_pnl = df['Net_INR'].sum()
    win_rate = (df['Net_Pts'] > 0).mean() * 100
    avg_move = df['Raw_Pts'].mean()
    
    print(f"Trades: {len(df)}")
    print(f"Avg Underlying Move: {avg_move:.2f} pts")
    print(f"Total Net PnL (Futures): ₹{total_pnl:,.2f}")
    print(f"Win Rate: {win_rate:.1f}%")
    print("----------------------------------------------")
    
    if total_pnl < 0:
        print("🚨 DIAGNOSIS: The FUTURES signal itself lost money today.")
        print("   This means the 'Kinetic Brain' failed to predict direction on this day.")
        print("   Options math cannot fix a bad underlying signal.")
    else:
        print("✅ DIAGNOSIS: The FUTURES signal made money!")
        print("   If Options lost money, the spread/theta cost is too high.")

# ==========================================
# 6. DIRECTIONAL SCALPER ENGINE
# ==========================================
def run_directional_engine(trades_csv, option_map):
    print("\n4. Running Directional Call Simulation...")
    
    trades_df = pd.read_csv(trades_csv)
    trades_df['Entry_Time'] = pd.to_datetime(trades_df['Entry_Time'])
    trades_df['Exit_Time'] = pd.to_datetime(trades_df['Exit_Time'])
    
    results = []
    
    for i, row in trades_df.iterrows():
        entry_ts = row['Entry_Time']
        exit_ts = row['Exit_Time']
        K = int(row['ATM_Strike'])
        
        # We only need the CE file
        ce_path = option_map.get((K, 'CE'))
        if not ce_path:
            # print(f"   Skip Trade {i}: Missing CE file for Strike {K}")
            continue
            
        try:
            ce_df = pd.read_parquet(ce_path)
            if "DateTime" not in ce_df.columns:
                ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))
            ce_df.set_index("DateTime", inplace=True)
            if "LastTradedPrice" in ce_df.columns: ce_df["LTP"] = ce_df["LastTradedPrice"]
            ce_df = ce_df.sort_index()
        except Exception as e:
            print(f"   Error reading parquet: {e}")
            continue

        # Get Precise Entry/Exit Prices
        trade_data = ce_df.loc[entry_ts:exit_ts]
        if trade_data.empty:
            continue
            
        # ENTRY LOGIC (Price + Cost)
        entry_price_raw = trade_data.iloc[0]['LTP']
        entry_price_net = entry_price_raw + ENTRY_COST_PER_LEG
        
        # EXIT LOGIC (Price - Cost)
        exit_price_raw = trade_data.iloc[-1]['LTP']
        exit_price_net = exit_price_raw - EXIT_COST_PER_LEG
        
        # PnL Calculation
        pnl_points = exit_price_net - entry_price_net
        
        results.append({
            "Entry_Time": entry_ts,
            "Exit_Time": exit_ts,
            "Strike": K,
            "CE_Entry_Raw": entry_price_raw,
            "CE_Exit_Raw": exit_price_raw,
            "PnL_Points": pnl_points,
            "PnL_INR": pnl_points * LOT_SIZE
        })

    # Save Results
    res_df = pd.DataFrame(results)
    final_csv = os.path.join(BASE_FOLDER, "directional_scalping_results.csv")
    res_df.to_csv(final_csv, index=False)
    
    print("\n--- OPTION RESULTS ---")
    print(f"Total Trades: {len(res_df)}")
    if not res_df.empty:
        print(f"Total Net PnL: ₹{res_df['PnL_INR'].sum():,.2f}")
        print(f"Win Rate: {(res_df['PnL_Points'] > 0).mean()*100:.1f}%")
        print(f"Avg PnL per Trade: ₹{res_df['PnL_INR'].mean():,.2f}")
    else:
        print("No valid trades found.")
    print(f"Saved to: {final_csv}")

# ==========================================
# 7. MAIN EXECUTION
# ==========================================
def main():
    # A. Get Futures
    fut_csv = download_futures_for_day(YEAR, MONTH, DAY, SYMBOL, FUT_TS)
    
    # B. Generate Signals
    trades_csv = build_kinetic_trades(fut_csv)
    
    # C. RUN FUTURES BENCHMARK (The Truth Serum)
    run_futures_benchmark(trades_csv)
    
    # D. Identify Needed Options
    trades_df = pd.read_csv(trades_csv)
    if trades_df.empty:
        print("No trades generated.")
        return
        
    needed_strikes = trades_df["ATM_Strike"].unique().astype(int).tolist()
    expiry_code = find_nearest_expiry(YEAR, MONTH, DAY, SYMBOL)
    print(f"Target Expiry: {expiry_code}")
    
    # E. Get Options (ONLY CE)
    opt_map = download_specific_options(YEAR, MONTH, DAY, SYMBOL, expiry_code, needed_strikes)
    
    # F. Run Engine
    run_directional_engine(trades_csv, opt_map)

if __name__ == "__main__":
    main()

--- RUNNING DEEP ITM CALL SCALPER (DELTA ~0.85) FOR 20-11-2025 ---
1. Downloading Futures: year=2025/month=11/day=20/Futures/NIFTY/NIFTY25NOVFUT.parquet


/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/2614609573.py:77: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str), errors="coerce")


2. Generating Kinetic Signals...
   Generated 18 trades.

--- CONTROL GROUP: RAW FUTURES PERFORMANCE ---
Trades: 18
Avg Underlying Move: 4.16 pts
Total Net PnL (Futures): ₹4,260.00
Win Rate: 66.7%
----------------------------------------------
✅ DIAGNOSIS: The FUTURES signal made money!
   If Options lost money, the spread/theta cost is too high.
Target Expiry: 25D02
3. Downloading Options for Strikes: [25950, 25900, 26000, 26050]

4. Running Directional Call Simulation...


/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/2614609573.py:273: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))
/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/2614609573.py:273: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))
/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/2614609573.py:273: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datet


--- OPTION RESULTS ---
Total Trades: 18
Total Net PnL: ₹-4,522.50
Win Rate: 50.0%
Avg PnL per Trade: ₹-251.25
Saved to: data/2025-11-20/directional_scalping_results.csv


/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/2614609573.py:273: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))
/var/folders/4r/hrxj1wmn5pz2tdgcm9ppqckh0000gn/T/ipykernel_47008/2614609573.py:273: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S.%f format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  ce_df["DateTime"] = pd.to_datetime(ce_df["Date"].astype(str) + " " + ce_df["Time"].astype(str))


In [ ]:
import boto3
import pandas as pd
import numpy as np
import os
import math
import datetime
from collections import deque

# ==========================================
# 1. CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
DAY = 24
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"  # Trading symbol of futures on that date

# Strategy Parameters
LOT_SIZE = 75
RISK_FREE_RATE = 0.0          

# COST PARAMETERS (FUTURES ONLY)
# Nifty Futures typically have very low spread (0.05-0.10). 
# We assume 1.0 point TOTAL round-trip cost (Slippage + Brokerage + Taxes)
# This is a realistic conservative estimate for scalping.
FUTURES_ROUND_TRIP_COST = 1.0 

BASE_FOLDER = f"data/{YEAR}-{MONTH:02d}-{DAY:02d}/"
os.makedirs(BASE_FOLDER, exist_ok=True)

print(f"--- RUNNING LONG/SHORT FUTURES SCALPER FOR {DAY}-{MONTH}-{YEAR} ---")

# ==========================================
# 2. S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation: kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]: keys.append(obj["Key"])
        if resp.get("IsTruncated"): continuation = resp.get("NextContinuationToken")
        else: break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return # Skip if already exists
    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=BUCKET, Key=key)
    with open(local_path, "wb") as f:
        f.write(obj["Body"].read())

# ==========================================
# 3. FUTURES & KINETIC LOGIC
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(BASE_FOLDER, "FUT.parquet")
    print(f"1. Downloading Futures: {key}")
    download_parquet_to_path(key, local_path)
    
    # Convert to CSV for easier processing logic
    df = pd.read_parquet(local_path)
    # Fix Date Parsing Warning by enforcing format or dayfirst
    df["DateTime"] = pd.to_datetime(
        df["Date"].astype(str) + " " + df["Time"].astype(str), 
        dayfirst=True, 
        errors="coerce"
    )
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Handle Volume vs OpenInterest naming variations
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    csv_path = os.path.join(BASE_FOLDER, "FUT.csv")
    df.to_csv(csv_path, index=False)
    return csv_path

class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=1200):
        self.buf = deque(maxlen=50)
        self.in_trade = False
        self.entry_time = None
        self.threshold = threshold
        self.hold_seconds = hold_seconds

    def tick(self, ts, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0

        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1 # EXIT
            return 0 # HOLD

        arr = np.array(self.buf)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        disp = abs(prices[-1] - prices[0])
        score = traded_vol / (disp + 0.05)

        # DIRECTIONAL LOGIC ADDED:
        # Check if the price trend in the buffer is UP or DOWN
        price_change = prices[-1] - prices[0]

        if score > self.threshold:
            self.in_trade = True
            self.entry_time = ts
            
            # Determine Direction
            if price_change > 0:
                return 1 # LONG SIGNAL
            else:
                return 2 # SHORT SIGNAL

        return 0

def build_kinetic_trades(fut_csv):
    print("2. Generating Kinetic Signals...")
    df = pd.read_csv(fut_csv)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    
    brain = KineticBrain()
    trades = []
    active = None

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        sig = brain.tick(ts, ltp, vol)

        # START TRADE
        if sig == 1 or sig == 2:
            active = {
                "Date": ts.date(),
                "Entry_Time": ts,
                "Entry_LTP": ltp,
                "Side": "LONG" if sig == 1 else "SHORT"
            }
        
        # END TRADE
        elif sig == -1 and active is not None:
            active["Exit_Time"] = ts
            active["Exit_LTP"] = ltp
            trades.append(active)
            active = None

    trades_df = pd.DataFrame(trades)
    out_path = os.path.join(BASE_FOLDER, "kinetic_trades.csv")
    trades_df.to_csv(out_path, index=False)
    print(f"   Generated {len(trades_df)} trades.")
    return out_path

# ==========================================
# 4. FUTURES SIMULATION ENGINE
# ==========================================
def run_futures_simulation(trades_csv):
    print("\n3. Running Pure Futures Simulation...")
    df = pd.read_csv(trades_csv)
    if df.empty:
        print("No trades found.")
        return

    # CALCULATE PNL BASED ON DIRECTION
    # Long: Exit - Entry
    # Short: Entry - Exit
    df['Raw_Pts'] = np.where(
        df['Side'] == 'LONG', 
        df['Exit_LTP'] - df['Entry_LTP'], 
        df['Entry_LTP'] - df['Exit_LTP']
    )
    
    # Futures Costs: Fixed per round trip (Slippage + Comm)
    df['Net_Pts'] = df['Raw_Pts'] - FUTURES_ROUND_TRIP_COST
    df['Net_INR'] = df['Net_Pts'] * LOT_SIZE
    
    total_pnl = df['Net_INR'].sum()
    win_rate = (df['Net_Pts'] > 0).mean() * 100
    avg_raw = df['Raw_Pts'].mean()
    avg_net = df['Net_Pts'].mean()
    
    # Breakdown Long vs Short
    longs = df[df['Side'] == 'LONG']
    shorts = df[df['Side'] == 'SHORT']
    
    # Save Results
    final_csv = os.path.join(BASE_FOLDER, "futures_scalping_results.csv")
    df.to_csv(final_csv, index=False)
    
    print("\n--- FUTURES RESULTS ---")
    print(f"Total Trades:       {len(df)}")
    print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
    print(f"Win Rate:           {win_rate:.1f}%")
    print(f"Avg Net PnL:        {avg_net:.2f} pts")
    print("-" * 30)
    print(f"Longs: {len(longs)} | PnL: ₹{longs['Net_INR'].sum():,.2f}")
    print(f"Shorts: {len(shorts)} | PnL: ₹{shorts['Net_INR'].sum():,.2f}")
    print(f"Saved to:           {final_csv}")

# ==========================================
# 5. MAIN EXECUTION
# ==========================================
def main():
    # A. Get Futures
    fut_csv = download_futures_for_day(YEAR, MONTH, DAY, SYMBOL, FUT_TS)
    
    # B. Generate Signals
    trades_csv = build_kinetic_trades(fut_csv)
    
    # C. Run Simulation
    run_futures_simulation(trades_csv)

if __name__ == "__main__":
    main()

--- RUNNING LONG/SHORT FUTURES SCALPER FOR 24-11-2025 ---
1. Downloading Futures: year=2025/month=11/day=24/Futures/NIFTY/NIFTY25NOVFUT.parquet
2. Generating Kinetic Signals...
   Generated 17 trades.

3. Running Pure Futures Simulation...

--- FUTURES RESULTS ---
Total Trades:       17
Total Net PnL:      ₹1,215.00
Win Rate:           64.7%
Avg Net PnL:        0.95 pts
------------------------------
Longs: 4 | PnL: ₹-4,057.50
Shorts: 13 | PnL: ₹5,272.50
Saved to:           data/2025-11-24/futures_scalping_results.csv


In [ ]:
import boto3
import pandas as pd
import numpy as np
import os
import math
import datetime
from collections import deque

# ==========================================
# 1. CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"  # Trading symbol of futures on that date

# Strategy Parameters
LOT_SIZE = 75
RISK_FREE_RATE = 0.0          

# COST PARAMETERS (FUTURES ONLY)
# Nifty Futures typically have very low spread (0.05-0.10). 
# We assume 1.0 point TOTAL round-trip cost (Slippage + Brokerage + Taxes)
# This is a realistic conservative estimate for scalping.
FUTURES_ROUND_TRIP_COST = 1.0 

BASE_FOLDER = f"data/{YEAR}-{MONTH:02d}-{DAY:02d}/"
os.makedirs(BASE_FOLDER, exist_ok=True)

print(f"--- RUNNING LONG/SHORT FUTURES SCALPER FOR {DAY}-{MONTH}-{YEAR} ---")

# ==========================================
# 2. S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation: kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]: keys.append(obj["Key"])
        if resp.get("IsTruncated"): continuation = resp.get("NextContinuationToken")
        else: break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return # Skip if already exists
    s3 = boto3.client("s3")
    obj = s3.get_object(Bucket=BUCKET, Key=key)
    with open(local_path, "wb") as f:
        f.write(obj["Body"].read())

# ==========================================
# 3. FUTURES & KINETIC LOGIC
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(BASE_FOLDER, "FUT.parquet")
    print(f"1. Downloading Futures: {key}")
    download_parquet_to_path(key, local_path)
    
    # Convert to CSV for easier processing logic
    df = pd.read_parquet(local_path)
    # Fix Date Parsing Warning by enforcing format or dayfirst
    df["DateTime"] = pd.to_datetime(
        df["Date"].astype(str) + " " + df["Time"].astype(str), 
        dayfirst=True, 
        errors="coerce"
    )
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Handle Volume vs OpenInterest naming variations
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    csv_path = os.path.join(BASE_FOLDER, "FUT.csv")
    df.to_csv(csv_path, index=False)
    return csv_path

class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=1200):
        self.buf = deque(maxlen=50)
        self.in_trade = False
        self.entry_time = None
        self.threshold = threshold
        self.hold_seconds = hold_seconds

    def tick(self, ts, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0

        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1 # EXIT
            return 0 # HOLD

        arr = np.array(self.buf)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        disp = abs(prices[-1] - prices[0])
        score = traded_vol / (disp + 0.05)

        # DIRECTIONAL LOGIC (VWAP BASED):
        # We calculate the Volume Weighted Average Price of the current buffer.
        # If Current Price > VWAP -> Buyers are in control -> LONG
        # If Current Price < VWAP -> Sellers are in control -> SHORT
        
        # Calculate tick volumes (sanitize negative diffs)
        tick_vols = np.where(vol_diff < 0, 0, vol_diff)
        
        if np.sum(tick_vols) == 0:
             # Fallback to simple displacement if no volume
             direction = 1 if prices[-1] > prices[0] else 2
        else:
            # Match prices to volume (prices[1:] aligns with diff)
            trade_prices = prices[1:] 
            vwap = np.average(trade_prices, weights=tick_vols)
            
            if prices[-1] > vwap:
                direction = 1 # LONG
            else:
                direction = 2 # SHORT

        if score > self.threshold:
            self.in_trade = True
            self.entry_time = ts
            return direction

        return 0

def build_kinetic_trades(fut_csv):
    print("2. Generating Kinetic Signals...")
    df = pd.read_csv(fut_csv)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    
    brain = KineticBrain()
    trades = []
    active = None

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        sig = brain.tick(ts, ltp, vol)

        # START TRADE
        if sig == 1 or sig == 2:
            active = {
                "Date": ts.date(),
                "Entry_Time": ts,
                "Entry_LTP": ltp,
                "Side": "LONG" if sig == 1 else "SHORT"
            }
        
        # END TRADE
        elif sig == -1 and active is not None:
            active["Exit_Time"] = ts
            active["Exit_LTP"] = ltp
            trades.append(active)
            active = None

    trades_df = pd.DataFrame(trades)
    out_path = os.path.join(BASE_FOLDER, "kinetic_trades.csv")
    trades_df.to_csv(out_path, index=False)
    print(f"   Generated {len(trades_df)} trades.")
    return out_path

# ==========================================
# 4. FUTURES SIMULATION ENGINE
# ==========================================
def run_futures_simulation(trades_csv):
    print("\n3. Running Pure Futures Simulation...")
    df = pd.read_csv(trades_csv)
    if df.empty:
        print("No trades found.")
        return

    # CALCULATE PNL BASED ON DIRECTION
    # Long: Exit - Entry
    # Short: Entry - Exit
    df['Raw_Pts'] = np.where(
        df['Side'] == 'LONG', 
        df['Exit_LTP'] - df['Entry_LTP'], 
        df['Entry_LTP'] - df['Exit_LTP']
    )
    
    # Futures Costs: Fixed per round trip (Slippage + Comm)
    df['Net_Pts'] = df['Raw_Pts'] - FUTURES_ROUND_TRIP_COST
    df['Net_INR'] = df['Net_Pts'] * LOT_SIZE
    
    total_pnl = df['Net_INR'].sum()
    win_rate = (df['Net_Pts'] > 0).mean() * 100
    avg_raw = df['Raw_Pts'].mean()
    avg_net = df['Net_Pts'].mean()
    
    # Breakdown Long vs Short
    longs = df[df['Side'] == 'LONG']
    shorts = df[df['Side'] == 'SHORT']
    
    # Save Results
    final_csv = os.path.join(BASE_FOLDER, "futures_scalping_results.csv")
    df.to_csv(final_csv, index=False)
    
    print("\n--- FUTURES RESULTS ---")
    print(f"Total Trades:       {len(df)}")
    print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
    print(f"Win Rate:           {win_rate:.1f}%")
    print(f"Avg Net PnL:        {avg_net:.2f} pts")
    print("-" * 30)
    print(f"Longs: {len(longs)} | PnL: ₹{longs['Net_INR'].sum():,.2f}")
    print(f"Shorts: {len(shorts)} | PnL: ₹{shorts['Net_INR'].sum():,.2f}")
    print(f"Saved to:           {final_csv}")

# ==========================================
# 5. MAIN EXECUTION
# ==========================================
def main():
    # A. Get Futures
    fut_csv = download_futures_for_day(YEAR, MONTH, DAY, SYMBOL, FUT_TS)
    
    # B. Generate Signals
    trades_csv = build_kinetic_trades(fut_csv)
    
    # C. Run Simulation
    run_futures_simulation(trades_csv)

if __name__ == "__main__":
    main()

--- RUNNING LONG/SHORT FUTURES SCALPER FOR 24-11-2025 ---
1. Downloading Futures: year=2025/month=11/day=24/Futures/NIFTY/NIFTY25NOVFUT.parquet
2. Generating Kinetic Signals...
   Generated 17 trades.

3. Running Pure Futures Simulation...

--- FUTURES RESULTS ---
Total Trades:       17
Total Net PnL:      ₹2,760.00
Win Rate:           52.9%
Avg Net PnL:        2.16 pts
------------------------------
Longs: 10 | PnL: ₹-3,735.00
Shorts: 7 | PnL: ₹6,495.00
Saved to:           data/2025-11-24/futures_scalping_results.csv


In [ ]:
import boto3
import pandas as pd
import numpy as np
import os
import math
import datetime
from collections import deque

# ==========================================
# 1. CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 10
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25OCTFUT"  # Trading symbol of futures on that date

# Strategy Parameters
LOT_SIZE = 75
RISK_FREE_RATE = 0.0          

# Risk Management
DAILY_MAX_LOSS = -3000.0  # Stop trading if daily PnL hits -3000

# COST PARAMETERS (FUTURES ONLY)
FUTURES_ROUND_TRIP_COST = 1.0 

# Global Data Path (will be dynamic per day)
DATA_ROOT = "data/"

print(f"--- RUNNING MONTHLY FUTURES SCALPER (WITH RISK MGMT) FOR {MONTH}-{YEAR} ---")

# ==========================================
# 2. S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation: kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]: keys.append(obj["Key"])
        if resp.get("IsTruncated"): continuation = resp.get("NextContinuationToken")
        else: break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True # Skip if already exists
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except s3.exceptions.NoSuchKey:
        return False
    except Exception as e:
        print(f"Error downloading {key}: {e}")
        return False

# ==========================================
# 3. FUTURES & KINETIC LOGIC
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    
    success = download_parquet_to_path(key, local_path)
    if not success:
        return None
    
    # Convert to CSV for easier processing logic
    try:
        df = pd.read_parquet(local_path)
    except Exception:
        return None

    # Fix Date Parsing
    df["DateTime"] = pd.to_datetime(
        df["Date"].astype(str) + " " + df["Time"].astype(str), 
        dayfirst=True, 
        errors="coerce"
    )
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    # Handle Volume vs OpenInterest naming variations
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    csv_path = os.path.join(day_folder, "FUT.csv")
    df.to_csv(csv_path, index=False)
    return csv_path

class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=1200):
        self.buf = deque(maxlen=50)
        self.in_trade = False
        self.entry_time = None
        self.threshold = threshold
        self.hold_seconds = hold_seconds

    def tick(self, ts, ltp, vol):
        self.buf.append((ltp, vol))
        if len(self.buf) < 50: return 0

        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1 # EXIT
            return 0 # HOLD

        arr = np.array(self.buf)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        disp = abs(prices[-1] - prices[0])
        score = traded_vol / (disp + 0.05)

        # DIRECTIONAL LOGIC (VWAP BASED)
        tick_vols = np.where(vol_diff < 0, 0, vol_diff)
        
        if np.sum(tick_vols) == 0:
             # Fallback to simple displacement if no volume
             direction = 1 if prices[-1] > prices[0] else 2
        else:
            trade_prices = prices[1:] 
            vwap = np.average(trade_prices, weights=tick_vols)
            
            if prices[-1] > vwap:
                direction = 1 # LONG
            else:
                direction = 2 # SHORT

        if score > self.threshold:
            self.in_trade = True
            self.entry_time = ts
            return direction

        return 0

def build_kinetic_trades(fut_csv, day_folder):
    df = pd.read_csv(fut_csv)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    
    brain = KineticBrain()
    trades = []
    active = None

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        sig = brain.tick(ts, ltp, vol)

        # START TRADE
        if sig == 1 or sig == 2:
            active = {
                "Date": ts.date(),
                "Entry_Time": ts,
                "Entry_LTP": ltp,
                "Side": "LONG" if sig == 1 else "SHORT"
            }
        
        # END TRADE
        elif sig == -1 and active is not None:
            active["Exit_Time"] = ts
            active["Exit_LTP"] = ltp
            trades.append(active)
            active = None

    trades_df = pd.DataFrame(trades)
    out_path = os.path.join(day_folder, "kinetic_trades.csv")
    trades_df.to_csv(out_path, index=False)
    return trades_df

# ==========================================
# 4. FUTURES SIMULATION ENGINE (WITH RISK)
# ==========================================
def calculate_pnl_and_risk(df):
    if df.empty: return df

    # 1. Calculate Raw PnL for every trade
    df['Raw_Pts'] = np.where(
        df['Side'] == 'LONG', 
        df['Exit_LTP'] - df['Entry_LTP'], 
        df['Entry_LTP'] - df['Exit_LTP']
    )
    
    df['Net_Pts'] = df['Raw_Pts'] - FUTURES_ROUND_TRIP_COST
    df['Net_INR'] = df['Net_Pts'] * LOT_SIZE

    # 2. Apply Daily Stop Loss (Sequential Check)
    cumulative_pnl = df['Net_INR'].cumsum()
    
    # Find the trade where we breached the limit
    breach_mask = cumulative_pnl < DAILY_MAX_LOSS
    
    if breach_mask.any():
        # Get the index of the first breach
        first_breach_idx = breach_mask.idxmax()
        # We allow the trade that caused the breach to finish, but no more after that
        # So we keep trades up to first_breach_idx (inclusive)
        
        # NOTE: If idxmax returns the first True, slicing usually excludes the end if using standard python range
        # But loc/iloc is inclusive/exclusive differently.
        # Let's use simple filtering.
        
        # Find the integer location of the breach
        locs = np.where(breach_mask)[0]
        stop_loc = locs[0]
        
        # Keep only trades up to this point
        df = df.iloc[:stop_loc+1].copy()
        
        # Mark the last trade as the trigger
        print(f"   [RISK MGMT] Daily Stop Loss Hit! Stopped after trade #{stop_loc+1}. PnL: {df['Net_INR'].sum()}")

    return df

# ==========================================
# 5. MAIN EXECUTION
# ==========================================
def main():
    all_trades = []
    
    # Loop through all days of November
    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)
        
        print(f"\nProcessing {day}-{MONTH}-{YEAR}...")
        
        # A. Get Futures
        fut_csv = download_futures_for_day(YEAR, MONTH, day, SYMBOL, FUT_TS, day_folder)
        
        if not fut_csv:
            print(f"   No data found (Weekend/Holiday or Missing).")
            continue
            
        # B. Generate Signals
        trades_df = build_kinetic_trades(fut_csv, day_folder)
        
        if not trades_df.empty:
            # C. Calculate PnL with Risk Management
            trades_df = calculate_pnl_and_risk(trades_df)
            
            # Save daily result
            daily_csv = os.path.join(day_folder, "futures_results.csv")
            trades_df.to_csv(daily_csv, index=False)
            
            pnl = trades_df['Net_INR'].sum()
            print(f"   Trades: {len(trades_df)} | PnL: ₹{pnl:,.2f}")
            
            all_trades.append(trades_df)
        else:
            print("   No trades generated.")
    
    # D. Aggregate Monthly Results
    if all_trades:
        full_df = pd.concat(all_trades, ignore_index=True)
        
        total_pnl = full_df['Net_INR'].sum()
        win_rate = (full_df['Net_Pts'] > 0).mean() * 100
        avg_net = full_df['Net_Pts'].mean()
        
        longs = full_df[full_df['Side'] == 'LONG']
        shorts = full_df[full_df['Side'] == 'SHORT']
        
        print("\n" + "="*40)
        print(f" MONTHLY SUMMARY: {MONTH}-{YEAR}")
        print("="*40)
        print(f"Total Trading Days: {full_df['Date'].nunique()}")
        print(f"Total Trades:       {len(full_df)}")
        print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
        print(f"Win Rate:           {win_rate:.1f}%")
        print(f"Avg Net PnL/Trade:  {avg_net:.2f} pts")
        print("-" * 30)
        print(f"Longs:  {len(longs)} | PnL: ₹{longs['Net_INR'].sum():,.2f}")
        print(f"Shorts: {len(shorts)} | PnL: ₹{shorts['Net_INR'].sum():,.2f}")
        
        final_csv = f"data/nov_{YEAR}_full_results.csv"
        full_df.to_csv(final_csv, index=False)
        print(f"\nFull report saved to: {final_csv}")
    else:
        print("\nNo trades found for the entire month.")

if __name__ == "__main__":
    main()

--- RUNNING MONTHLY FUTURES SCALPER (WITH RISK MGMT) FOR 10-2025 ---

Processing 1-10-2025...
   No data found (Weekend/Holiday or Missing).

Processing 2-10-2025...
   No data found (Weekend/Holiday or Missing).

Processing 3-10-2025...
   [RISK MGMT] Daily Stop Loss Hit! Stopped after trade #4. PnL: -3375.000000000273
   Trades: 4 | PnL: ₹-3,375.00

Processing 4-10-2025...
   No data found (Weekend/Holiday or Missing).

Processing 5-10-2025...
   No data found (Weekend/Holiday or Missing).

Processing 6-10-2025...
   [RISK MGMT] Daily Stop Loss Hit! Stopped after trade #3. PnL: -4312.5
   Trades: 3 | PnL: ₹-4,312.50

Processing 7-10-2025...
   [RISK MGMT] Daily Stop Loss Hit! Stopped after trade #2. PnL: -5790.000000000055
   Trades: 2 | PnL: ₹-5,790.00

Processing 8-10-2025...
   Trades: 17 | PnL: ₹-1,492.50

Processing 9-10-2025...
   [RISK MGMT] Daily Stop Loss Hit! Stopped after trade #4. PnL: -3007.5000000001637
   Trades: 4 | PnL: ₹-3,007.50

Processing 10-10-2025...
   [RISK M

In [ ]:
"""
AWS FUTURES SCALPER WITH ROBUST DIRECTIONAL SIGNAL
===================================================
Kinetic Energy + Multi-Signal Direction Classifier
"""

import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 11
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"

# Strategy Parameters
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500
HOLD_SECONDS = 1200  # 20 minutes

# Risk Management
DAILY_MAX_LOSS = -3000.0

# Costs
FUTURES_ROUND_TRIP_COST = 1.0

DATA_ROOT = "data/"

print(f"\n{'='*60}")
print(f"ROBUST FUTURES SCALPER - {MONTH}/{YEAR}")
print(f"{'='*60}\n")


# ==========================================
# S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation:
            kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]:
                keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            continuation = resp.get("NextContinuationToken")
        else:
            break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except Exception as e:
        return False


# ==========================================
# ROBUST DIRECTIONAL ENGINE
# ==========================================
class RobustDirectionalEngine:
    """Multi-signal directional classifier"""
    
    def __init__(self, buffer_size=50):
        self.buffer_size = buffer_size
        self.price_buffer = deque(maxlen=buffer_size)
        self.volume_buffer = deque(maxlen=buffer_size)
        
    def add_tick(self, ltp, volume):
        self.price_buffer.append(ltp)
        self.volume_buffer.append(volume)
    
    def calculate_direction(self):
        """Returns: 1 (LONG), 2 (SHORT), 0 (NEUTRAL)"""
        if len(self.price_buffer) < self.buffer_size:
            return 0
        
        prices = np.array(self.price_buffer)
        volumes = np.array(self.volume_buffer)
        
        # Calculate all signals
        signals = {
            'vwap_momentum': self._vwap_momentum_signal(prices, volumes),
            'aggressor_flow': self._aggressor_flow_signal(prices, volumes),
            'price_acceleration': self._price_acceleration_signal(prices, volumes),
            'volume_momentum': self._volume_momentum_signal(prices, volumes)
        }
        
        weights = {
            'vwap_momentum': 0.35,
            'aggressor_flow': 0.30,
            'price_acceleration': 0.20,
            'volume_momentum': 0.15
        }
        
        # Weighted combination
        direction_score = sum(signals[k] * weights[k] for k in signals.keys())
        
        # Decision thresholds
        if direction_score > 0.15:
            return 1  # LONG
        elif direction_score < -0.15:
            return 2  # SHORT
        else:
            return 0  # NEUTRAL (skip trade)
    
    def _vwap_momentum_signal(self, prices, volumes):
        """Compare price to VWAP"""
        vol_deltas = np.diff(volumes)
        vol_deltas = np.where(vol_deltas > 0, vol_deltas, 0)
        
        if vol_deltas.sum() == 0:
            return 0.0
        
        vwap = np.average(prices[1:], weights=vol_deltas)
        current = prices[-1]
        deviation = (current - vwap) / vwap
        
        return np.tanh(deviation * 300)
    
    def _aggressor_flow_signal(self, prices, volumes):
        """Lee-Ready trade classification"""
        mid = (prices[:-1] + prices[1:]) / 2
        trade_signs = np.zeros(len(prices) - 1)
        
        for i in range(len(prices) - 1):
            if prices[i+1] > mid[i]:
                trade_signs[i] = 1
            elif prices[i+1] < mid[i]:
                trade_signs[i] = -1
            else:
                if i > 0:
                    trade_signs[i] = np.sign(prices[i+1] - prices[i])
        
        vol_deltas = np.diff(volumes)
        vol_deltas = np.where(vol_deltas > 0, vol_deltas, 0)
        
        if vol_deltas.sum() == 0:
            return 0.0
        
        net_flow = (trade_signs * vol_deltas).sum() / vol_deltas.sum()
        return np.clip(net_flow * 2, -1, 1)
    
    def _price_acceleration_signal(self, prices, volumes):
        """Detect momentum acceleration"""
        mid = len(prices) // 2
        first_half = prices[:mid]
        second_half = prices[mid:]
        
        mom_first = (first_half[-1] - first_half[0]) / first_half[0]
        mom_second = (second_half[-1] - second_half[0]) / second_half[0]
        
        acceleration = mom_second - mom_first
        return np.tanh(acceleration * 1000)
    
    def _volume_momentum_signal(self, prices, volumes):
        """Volume concentration + direction"""
        price_direction = prices[-1] - prices[0]
        
        if abs(price_direction) < 0.01:
            return 0.0
        
        vol_deltas = np.diff(volumes)
        vol_deltas = np.where(vol_deltas > 0, vol_deltas, 0)
        
        if vol_deltas.sum() == 0:
            return 0.0
        
        recent_vol = vol_deltas[-len(vol_deltas)//4:].sum()
        total_vol = vol_deltas.sum()
        
        concentration = recent_vol / total_vol
        strength = concentration * 2 - 0.5
        
        return np.sign(price_direction) * strength


# ==========================================
# ENHANCED KINETIC BRAIN
# ==========================================
class EnhancedKineticBrain:
    """Kinetic Energy + Robust Direction"""
    
    def __init__(self, threshold=37500, hold_seconds=1200):
        self.threshold = threshold
        self.hold_seconds = hold_seconds
        self.in_trade = False
        self.entry_time = None
        self.ke_buffer = deque(maxlen=50)
        self.directional = RobustDirectionalEngine(buffer_size=50)
        self.last_score = 0
    
    def tick(self, ts, ltp, volume):
        """Process tick and return signal"""
        self.ke_buffer.append((ltp, volume))
        self.directional.add_tick(ltp, volume)
        
        if len(self.ke_buffer) < 50:
            return 0
        
        # Exit check
        if self.in_trade:
            if (ts - self.entry_time).total_seconds() >= self.hold_seconds:
                self.in_trade = False
                return -1
            return 0
        
        # Calculate KE
        arr = np.array(self.ke_buffer)
        prices, vols = arr[:, 0], arr[:, 1]
        
        vol_diff = np.diff(vols)
        traded_vol = np.sum(np.where(vol_diff > 0, vol_diff, 0))
        displacement = abs(prices[-1] - prices[0])
        
        ke_score = traded_vol / (displacement + 0.05)
        self.last_score = ke_score
        
        # Entry: KE trigger + Direction
        if ke_score > self.threshold:
            direction = self.directional.calculate_direction()
            
            if direction > 0:
                self.in_trade = True
                self.entry_time = ts
                return direction
        
        return 0


# ==========================================
# DATA LOADING
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    
    success = download_parquet_to_path(key, local_path)
    if not success:
        return None
    
    try:
        df = pd.read_parquet(local_path)
    except Exception:
        return None

    df["DateTime"] = pd.to_datetime(
        df["Date"].astype(str) + " " + df["Time"].astype(str), 
        dayfirst=True, 
        errors="coerce"
    )
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    if "Volume" in df.columns:
        df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns:
        df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else:
        df["Volume"] = 0.0
    
    csv_path = os.path.join(day_folder, "FUT.csv")
    df.to_csv(csv_path, index=False)
    return csv_path

def build_kinetic_trades(fut_csv, day_folder):
    df = pd.read_csv(fut_csv)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    
    brain = EnhancedKineticBrain(threshold=KINETIC_THRESHOLD, hold_seconds=HOLD_SECONDS)
    
    trades = []
    active = None

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        
        signal = brain.tick(ts, ltp, vol)

        if signal == 1 or signal == 2:
            active = {
                "Date": ts.date(),
                "Entry_Time": ts,
                "Entry_LTP": ltp,
                "Side": "LONG" if signal == 1 else "SHORT",
                "KE_Score": brain.last_score
            }
        
        elif signal == -1 and active is not None:
            active["Exit_Time"] = ts
            active["Exit_LTP"] = ltp
            trades.append(active)
            active = None

    trades_df = pd.DataFrame(trades)
    out_path = os.path.join(day_folder, "kinetic_trades.csv")
    trades_df.to_csv(out_path, index=False)
    
    return trades_df


# ==========================================
# PNL CALCULATION WITH RISK MGMT
# ==========================================
def calculate_pnl_and_risk(df):
    if df.empty:
        return df

    df['Raw_Pts'] = np.where(
        df['Side'] == 'LONG', 
        df['Exit_LTP'] - df['Entry_LTP'], 
        df['Entry_LTP'] - df['Exit_LTP']
    )
    
    df['Net_Pts'] = df['Raw_Pts'] - FUTURES_ROUND_TRIP_COST
    df['Net_INR'] = df['Net_Pts'] * LOT_SIZE

    # Daily stop loss
    cumulative_pnl = df['Net_INR'].cumsum()
    breach_mask = cumulative_pnl < DAILY_MAX_LOSS
    
    if breach_mask.any():
        locs = np.where(breach_mask)[0]
        stop_loc = locs[0]
        df = df.iloc[:stop_loc+1].copy()
        print(f"   [STOP LOSS] Hit after trade #{stop_loc+1}. PnL: ₹{df['Net_INR'].sum():,.2f}")

    return df


# ==========================================
# MAIN EXECUTION
# ==========================================
def main():
    all_trades = []
    
    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)
        
        print(f"\n{day:02d}/{MONTH}/{YEAR}...", end=" ")
        
        fut_csv = download_futures_for_day(YEAR, MONTH, day, SYMBOL, FUT_TS, day_folder)
        
        if not fut_csv:
            print("No data")
            continue
        
        trades_df = build_kinetic_trades(fut_csv, day_folder)
        
        if not trades_df.empty:
            trades_df = calculate_pnl_and_risk(trades_df)
            
            daily_csv = os.path.join(day_folder, "futures_results.csv")
            trades_df.to_csv(daily_csv, index=False)
            
            pnl = trades_df['Net_INR'].sum()
            n_trades = len(trades_df)
            longs = len(trades_df[trades_df['Side'] == 'LONG'])
            shorts = len(trades_df[trades_df['Side'] == 'SHORT'])
            
            print(f"Trades: {n_trades} (L:{longs}/S:{shorts}) | PnL: ₹{pnl:,.0f}")
            
            all_trades.append(trades_df)
        else:
            print("No signals")
    
    # Monthly summary
    if all_trades:
        full_df = pd.concat(all_trades, ignore_index=True)
        
        total_pnl = full_df['Net_INR'].sum()
        win_rate = (full_df['Net_Pts'] > 0).mean() * 100
        avg_net = full_df['Net_Pts'].mean()
        
        longs = full_df[full_df['Side'] == 'LONG']
        shorts = full_df[full_df['Side'] == 'SHORT']
        
        print("\n" + "="*60)
        print(f"MONTHLY SUMMARY: {MONTH}/{YEAR}")
        print("="*60)
        print(f"Trading Days:       {full_df['Date'].nunique()}")
        print(f"Total Trades:       {len(full_df)}")
        print(f"  - Longs:          {len(longs)} (₹{longs['Net_INR'].sum():,.0f})")
        print(f"  - Shorts:         {len(shorts)} (₹{shorts['Net_INR'].sum():,.0f})")
        print(f"\nTotal Net PnL:      ₹{total_pnl:,.2f}")
        print(f"Avg PnL/Trade:      {avg_net:.2f} pts (₹{avg_net*LOT_SIZE:.2f})")
        print(f"Win Rate:           {win_rate:.1f}%")
        
        # Direction accuracy
        long_wr = (longs['Net_Pts'] > 0).mean() * 100 if len(longs) > 0 else 0
        short_wr = (shorts['Net_Pts'] > 0).mean() * 100 if len(shorts) > 0 else 0
        
        print(f"\nDirectional Accuracy:")
        print(f"  - Long Win Rate:  {long_wr:.1f}%")
        print(f"  - Short Win Rate: {short_wr:.1f}%")
        
        final_csv = f"data/nov_{YEAR}_robust_results.csv"
        full_df.to_csv(final_csv, index=False)
        print(f"\nResults saved: {final_csv}")
        print("="*60)
    else:
        print("\nNo trades for the month.")

if __name__ == "__main__":
    main()


ROBUST FUTURES SCALPER - 11/2025


01/11/2025... No data

02/11/2025... No data

03/11/2025... No data

04/11/2025...    [STOP LOSS] Hit after trade #1. PnL: ₹-5,527.50
Trades: 1 (L:1/S:0) | PnL: ₹-5,528

05/11/2025... No data

06/11/2025...    [STOP LOSS] Hit after trade #3. PnL: ₹-9,742.50
Trades: 3 (L:2/S:1) | PnL: ₹-9,743

07/11/2025...    [STOP LOSS] Hit after trade #1. PnL: ₹-4,665.00
Trades: 1 (L:1/S:0) | PnL: ₹-4,665

08/11/2025... No data

09/11/2025... No data

10/11/2025... No data

11/11/2025... Trades: 17 (L:11/S:6) | PnL: ₹7,612

12/11/2025...    [STOP LOSS] Hit after trade #9. PnL: ₹-3,862.50
Trades: 9 (L:3/S:6) | PnL: ₹-3,862

13/11/2025...    [STOP LOSS] Hit after trade #3. PnL: ₹-4,237.50
Trades: 3 (L:0/S:3) | PnL: ₹-4,238

14/11/2025... No data

15/11/2025... No data

16/11/2025... No data

17/11/2025... Trades: 16 (L:8/S:8) | PnL: ₹-75

18/11/2025... Trades: 16 (L:8/S:8) | PnL: ₹6,022

19/11/2025...    [STOP LOSS] Hit after trade #3. PnL: ₹-4,530.00
Trades: 3 (L:0/

gemini

In [ ]:
"""
AWS FUTURES SCALPER: EFFICIENCY HYBRID + 2-STRIKE GUARD
========================================================
Logic: 
1. Core: Efficiency Ratio (ER) + Supertrend (The Brain).
2. Defense (New): 
   - CONSECUTIVE LOSS LIMIT = 2. (If we lose twice in a row, we quit today).
   - Trailing Stop: Move to BE at +20 pts to kill risk early.
3. Goal: Cut the -4k days down to -1k days, keeping the +13k days intact.
"""

import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
YEAR = 2025
MONTH = 10
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25OCTFUT"

# Strategy Parameters
LOT_SIZE = 75
KINETIC_THRESHOLD = 37500    
MAX_HOLD_SECONDS = 1200      # 20 mins

# Efficiency Settings
ER_PERIOD = 50          
ER_TREND_THRESHOLD = 0.50  
ER_CHOP_THRESHOLD = 0.30   

# Supertrend Settings
ATR_PERIOD = 7
ATR_MULTIPLIER = 3.0

# Trade Management
STOP_LOSS_POINTS = 30.0      
TAKE_PROFIT_POINTS = 80.0    # Increased to let trailing work

# Trailing Settings (THE SHIELD)
BE_TRIGGER = 20.0       # At +20 pts, Move SL to Breakeven
TRAIL_TRIGGER = 40.0    # At +40 pts, Start Trailing
TRAIL_LOCK = 20.0       # Lock 20 pts profit

# Risk Management
DAILY_MAX_LOSS = -3000.0
CONSECUTIVE_LOSS_LIMIT = 2 # The "2-Strike" Rule

# Costs
FUTURES_ROUND_TRIP_COST = 1.0

print(f"\n{'='*60}")
print(f"EFFICIENCY HYBRID (2-STRIKE GUARD) - {MONTH}/{YEAR}")
print(f"Logic: ER Hybrid | Stop after {CONSECUTIVE_LOSS_LIMIT} losses | BE @ +{BE_TRIGGER}")
print(f"{'='*60}\n")


# ==========================================
# S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation:
            kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]:
                keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            continuation = resp.get("NextContinuationToken")
        else:
            break
    return keys

def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except Exception as e:
        return False


# ==========================================
# SUPERTREND INDICATOR
# ==========================================
class Supertrend:
    def __init__(self, period=7, multiplier=3.0):
        self.period = period
        self.multiplier = multiplier
        self.tr_buffer = deque(maxlen=period)
        self.prev_close = None
        self.trend = 1 
        self.upper_band = 0.0
        self.lower_band = 0.0
        self.supertrend = 0.0
        self.initialized = False
        
    def update(self, high, low, close):
        if self.prev_close is None:
            tr = high - low
        else:
            tr = max(high - low, abs(high - self.prev_close), abs(low - self.prev_close))
        
        self.tr_buffer.append(tr)
        self.prev_close = close
        
        if len(self.tr_buffer) < self.period:
            return 0 
            
        atr = np.mean(self.tr_buffer)
        mid = (high + low) / 2
        basic_upper = mid + (self.multiplier * atr)
        basic_lower = mid - (self.multiplier * atr)
        
        if not self.initialized:
            self.upper_band = basic_upper
            self.lower_band = basic_lower
            self.initialized = True
        
        if basic_upper < self.upper_band or self.prev_close > self.upper_band:
            self.upper_band = basic_upper
        if basic_lower > self.lower_band or self.prev_close < self.lower_band:
            self.lower_band = basic_lower
            
        if self.trend == 1 and close < self.lower_band:
            self.trend = -1
            self.supertrend = self.upper_band
        elif self.trend == -1 and close > self.upper_band:
            self.trend = 1
            self.supertrend = self.lower_band
        elif self.trend == 1:
            self.supertrend = self.lower_band
        elif self.trend == -1:
            self.supertrend = self.upper_band
            
        return self.trend


# ==========================================
# EFFICIENCY KINETIC BRAIN
# ==========================================
class EfficiencyKineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.ke_buffer = deque(maxlen=50)
        self.price_buffer = deque(maxlen=ER_PERIOD)
        self.st = Supertrend(period=ATR_PERIOD, multiplier=ATR_MULTIPLIER)
        self.last_score = 0
    
    def tick(self, ltp, volume):
        self.ke_buffer.append((ltp, volume))
        self.price_buffer.append(ltp)
        st_dir = self.st.update(ltp, ltp, ltp)
        
        if len(self.ke_buffer) < 50 or len(self.price_buffer) < ER_PERIOD:
            return 0
            
        # 1. Kinetic Calculation
        arr = np.array(self.ke_buffer)
        prices_ke = arr[:, 0]
        vols_ke = arr[:, 1]
        
        v_diff = np.diff(vols_ke)
        v_diff = np.where(v_diff < 0, 0, v_diff)
        traded_vol = np.sum(v_diff)
        displacement = abs(prices_ke[-1] - prices_ke[0])
        
        ke_score = traded_vol / (displacement + 0.05)
        self.last_score = ke_score
        
        # 2. Trigger
        if ke_score > self.threshold:
            
            # 3. Calculate Efficiency Ratio (ER)
            prices_er = np.array(self.price_buffer)
            net_change = abs(prices_er[-1] - prices_er[0])
            sum_changes = np.sum(np.abs(np.diff(prices_er)))
            
            if sum_changes == 0: 
                er = 1.0 
            else:
                er = net_change / sum_changes
                
            # 4. Hybrid Logic
            
            # CASE A: High Efficiency (TREND MODE) -> FOLLOW
            if er > ER_TREND_THRESHOLD:
                if st_dir == 1: return 1 # Follow Long
                if st_dir == -1: return 2 # Follow Short
                
            # CASE B: Low Efficiency (CHOP MODE) -> FADE
            elif er < ER_CHOP_THRESHOLD:
                if st_dir == 1: return 2 # Fade Long (Go Short)
                if st_dir == -1: return 1 # Fade Short (Go Long)
            
            # CASE C: Middle (NOISE) -> NO TRADE
            
        return 0


# ==========================================
# DATA LOADING
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    success = download_parquet_to_path(key, local_path)
    if not success: return None
    
    try:
        df = pd.read_parquet(local_path)
    except: return None

    df["DateTime"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str), dayfirst=True, errors="coerce")
    df["LTP"] = pd.to_numeric(df["LTP"], errors="coerce")
    df = df.dropna(subset=["DateTime", "LTP"]).sort_values("DateTime").reset_index(drop=True)
    
    if "Volume" in df.columns: df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns: df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else: df["Volume"] = 0.0
    
    return df

def process_day_trades(df, day_folder):
    brain = EfficiencyKineticBrain(threshold=KINETIC_THRESHOLD)
    completed_trades = []
    
    # Active Trade State
    in_trade = False
    entry_time = None
    entry_price = 0.0
    side = None 
    highest_pnl = 0.0 # Track max profit for trailing
    
    # Risk State
    daily_pnl = 0.0
    consecutive_losses = 0
    
    for _, row in df.iterrows():
        # GLOBAL STOPS
        if daily_pnl < DAILY_MAX_LOSS: break 
        if consecutive_losses >= CONSECUTIVE_LOSS_LIMIT: break # 2-Strike Out
            
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])
        
        # --- TRADE MANAGEMENT ---
        if in_trade:
            duration = (ts - entry_time).total_seconds()
            exit_signal = False
            exit_reason = ""
            
            # PnL Calc
            if side == "LONG":
                float_pnl = ltp - entry_price
            else:
                float_pnl = entry_price - ltp
            
            # Update Peak
            if float_pnl > highest_pnl: highest_pnl = float_pnl
            
            # Dynamic Stop Logic
            current_sl = STOP_LOSS_POINTS
            if highest_pnl >= TRAIL_TRIGGER: current_sl = -TRAIL_LOCK # Lock in profit
            elif highest_pnl >= BE_TRIGGER: current_sl = -1.0 # Breakeven
            
            # Exits
            if float_pnl <= -current_sl: 
                exit_signal = True
                exit_reason = "TRAIL/BE" if current_sl < 0 else "SL"
            elif float_pnl >= TAKE_PROFIT_POINTS: 
                exit_signal = True; exit_reason = "TP"
            elif duration >= MAX_HOLD_SECONDS:
                exit_signal = True; exit_reason = "TIME"
                
            if exit_signal:
                gross_pnl = float_pnl * LOT_SIZE
                cost = FUTURES_ROUND_TRIP_COST * LOT_SIZE
                net_pnl = gross_pnl - cost
                
                daily_pnl += net_pnl
                
                # Update Streak
                if net_pnl < 0: consecutive_losses += 1
                else: consecutive_losses = 0 # Reset on win
                
                completed_trades.append({
                    "Date": ts.date(),
                    "Entry_Time": entry_time,
                    "Side": side,
                    "Net_Pts": float_pnl - FUTURES_ROUND_TRIP_COST,
                    "Net_INR": net_pnl,
                    "Exit_Reason": exit_reason
                })
                in_trade = False
                highest_pnl = 0.0
                continue 

        # --- SIGNAL GENERATION ---
        if not in_trade:
            sig = brain.tick(ltp, vol)
            if sig == 1: 
                in_trade = True; entry_time = ts; entry_price = ltp; side = "LONG"
            elif sig == 2: 
                in_trade = True; entry_time = ts; entry_price = ltp; side = "SHORT"
        else:
            brain.tick(ltp, vol)

    trades_df = pd.DataFrame(completed_trades)
    return trades_df


# ==========================================
# MAIN EXECUTION
# ==========================================
def main():
    all_trades = []
    
    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)
        
        print(f"\n{day:02d}/{MONTH}/{YEAR}...", end=" ")
        
        df = download_futures_for_day(YEAR, MONTH, day, SYMBOL, FUT_TS, day_folder)
        
        if df is None or df.empty:
            print("No data")
            continue
        
        trades_df = process_day_trades(df, day_folder)
        
        if not trades_df.empty:
            daily_csv = os.path.join(day_folder, "futures_results.csv")
            trades_df.to_csv(daily_csv, index=False)
            
            pnl = trades_df['Net_INR'].sum()
            n_trades = len(trades_df)
            sl_hits = len(trades_df[trades_df['Exit_Reason'] == 'SL'])
            be_hits = len(trades_df[trades_df['Exit_Reason'] == 'TRAIL/BE'])
            streak_end = "YES" if pnl < DAILY_MAX_LOSS or len(trades_df) < 5 else "NO" # approx logic check
            
            # Check if stopped by streak
            # We can infer it: if consecutive losses at end is >= 2
            
            print(f"Trades: {n_trades} (SL:{sl_hits}/BE:{be_hits}) | PnL: ₹{pnl:,.0f}")
            if pnl < DAILY_MAX_LOSS: print(f"   [RISK] Global Stop Hit!")
            
            all_trades.append(trades_df)
        else:
            print("No trades")
    
    if all_trades:
        full_df = pd.concat(all_trades, ignore_index=True)
        
        total_pnl = full_df['Net_INR'].sum()
        win_rate = (full_df['Net_Pts'] > 0).mean() * 100
        
        longs = full_df[full_df['Side'] == 'LONG']
        shorts = full_df[full_df['Side'] == 'SHORT']
        
        print("\n" + "="*60)
        print(f"EFFICIENCY HYBRID (GUARDED) SUMMARY: {MONTH}/{YEAR}")
        print("="*60)
        print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
        print(f"Win Rate:           {win_rate:.1f}%")
        print(f"Total Trades:       {len(full_df)}")
        print(f"  - Longs:          {len(longs)} (₹{longs['Net_INR'].sum():,.0f})")
        print(f"  - Shorts:         {len(shorts)} (₹{shorts['Net_INR'].sum():,.0f})")
        
        final_csv = f"data/nov_{YEAR}_hybrid_guarded_results.csv"
        full_df.to_csv(final_csv, index=False)
        print(f"\nResults saved: {final_csv}")
    else:
        print("\nNo trades for the month.")

if __name__ == "__main__":
    main()


EFFICIENCY HYBRID (2-STRIKE GUARD) - 10/2025
Logic: ER Hybrid | Stop after 2 losses | BE @ +20.0


01/10/2025... No data

02/10/2025... No data

03/10/2025... Trades: 9 (SL:2/BE:2) | PnL: ₹-2,025

04/10/2025... No data

05/10/2025... No data

06/10/2025... Trades: 3 (SL:2/BE:1) | PnL: ₹-3,240
   [RISK] Global Stop Hit!

07/10/2025... Trades: 2 (SL:1/BE:1) | PnL: ₹-2,415

08/10/2025... Trades: 2 (SL:2/BE:0) | PnL: ₹-4,860
   [RISK] Global Stop Hit!

09/10/2025... Trades: 7 (SL:3/BE:3) | PnL: ₹-3,923
   [RISK] Global Stop Hit!

10/10/2025... Trades: 2 (SL:2/BE:0) | PnL: ₹-4,823
   [RISK] Global Stop Hit!

11/10/2025... No data

12/10/2025... No data

13/10/2025... Trades: 6 (SL:1/BE:3) | PnL: ₹1,988

14/10/2025... Trades: 15 (SL:1/BE:2) | PnL: ₹11,490

15/10/2025... Trades: 4 (SL:2/BE:0) | PnL: ₹-2,340

16/10/2025... Trades: 2 (SL:1/BE:1) | PnL: ₹-2,558

17/10/2025... Trades: 2 (SL:2/BE:0) | PnL: ₹-4,702
   [RISK] Global Stop Hit!

18/10/2025... No data

19/10/2025... No data

20/10/202

In [ ]:
"""
AWS FUTURES SCALPER: KINETIC DIRECTIONAL BRAIN
==============================================
Logic:
- Detect big kinetic bursts (volume / displacement) like your core bot
- On each burst, decide LONG / SHORT using:
    * Signed orderflow (uptick vs downtick volume)
    * Fast vs slow slopes (trend alignment)
    * Position in local range (are we at extremes?)
- Only trade when multiple signals agree (vote filter)
"""

import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"

YEAR = 2025
MONTH = 11
SYMBOL = "NIFTY"
FUT_TS = "NIFTY25NOVFUT"  # change if needed

# NIFTY Futures Settings
LOT_SIZE = 75

# Kinetic Directional Brain Params
KINETIC_THRESHOLD = 37500      # same spirit as your core bot
PRICE_WINDOW = 60              # ticks for local range
VOL_WINDOW = 60                # ticks for KE + signed vol
FAST_LOOKBACK = 10             # short slope window
SLOW_LOOKBACK = 30             # medium slope window
MIN_VOTES = 3                  # how strict we are (3+ recommended)

# Trade Management
MAX_HOLD_SECONDS = 900         # 15 min hold cap
STOP_LOSS_POINTS = 25.0        # per lot
TAKE_PROFIT_POINTS = 50.0      # per lot

# Costs
FUTURES_ROUND_TRIP_COST = 1.0  # in index points (both sides combined)

# Risk Management
DAILY_MAX_LOSS = -3000.0       # INR, hard stop per day

print(f"\n{'='*60}")
print(f"KINETIC DIRECTIONAL FUTURES SCALPER - {MONTH}/{YEAR}")
print(f"Symbol: {SYMBOL}, Contract: {FUT_TS}")
print(f"{'='*60}\n")


# ==========================================
# S3 UTILITIES
# ==========================================
def list_s3(prefix: str):
    s3 = boto3.client("s3")
    keys = []
    continuation = None
    while True:
        kwargs = {"Bucket": BUCKET, "Prefix": prefix}
        if continuation:
            kwargs["ContinuationToken"] = continuation
        resp = s3.list_objects_v2(**kwargs)
        if "Contents" in resp:
            for obj in resp["Contents"]:
                keys.append(obj["Key"])
        if resp.get("IsTruncated"):
            continuation = resp.get("NextContinuationToken")
        else:
            break
    return keys


def download_parquet_to_path(key: str, local_path: str):
    if os.path.exists(local_path):
        return True
    s3 = boto3.client("s3")
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        with open(local_path, "wb") as f:
            f.write(obj["Body"].read())
        return True
    except Exception:
        return False


# ==========================================
# KINETIC DIRECTIONAL BRAIN
# ==========================================
class KineticDirectionalBrain:
    """
    Directional overlay on top of your Kinetic burst.
    Uses:
      - Kinetic score (volume / displacement)
      - Signed orderflow (volume on upticks vs downticks)
      - Short/medium price slope
      - Local range position

    tick(ltp, volume) -> 
        0  = no trade
        +1 = LONG
        -1 = SHORT
    """
    def __init__(
        self,
        kinetic_threshold=37500,
        price_window=60,
        vol_window=60,
        fast_lookback=10,
        slow_lookback=30,
        min_votes=3
    ):
        self.kinetic_threshold = kinetic_threshold
        self.price_window = price_window
        self.vol_window = vol_window
        self.fast_lookback = fast_lookback
        self.slow_lookback = slow_lookback
        self.min_votes = min_votes

        self.price_buf = deque(maxlen=max(price_window, vol_window))
        self.vol_buf = deque(maxlen=vol_window)
        self.last_ke_score = 0.0

    def _lin_slope(self, arr):
        """Simple linear regression slope over index 0..n-1."""
        n = len(arr)
        if n < 2:
            return 0.0
        x = np.arange(n, dtype=float)
        y = np.asarray(arr, dtype=float)
        x_mean = x.mean()
        y_mean = y.mean()
        num = np.sum((x - x_mean) * (y - y_mean))
        den = np.sum((x - x_mean) ** 2) + 1e-9
        return num / den

    def tick(self, ltp, volume):
        """
        Call this for EVERY tick.

        Returns:
          0  = no signal
          +1 = LONG
          -1 = SHORT
        """
        ltp = float(ltp)
        volume = float(volume)

        self.price_buf.append(ltp)
        self.vol_buf.append(volume)

        if len(self.vol_buf) < self.vol_window or len(self.price_buf) < self.price_window:
            return 0

        prices = np.asarray(self.price_buf, dtype=float)
        vols = np.asarray(self.vol_buf, dtype=float)

        # ---------- 1) KINETIC SCORE ----------
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0.0)
        if trade_vol.sum() <= 0:
            return 0

        displacement = abs(prices[-1] - prices[-self.vol_window])
        ke_score = trade_vol.sum() / (displacement + 0.05)
        self.last_ke_score = ke_score

        if ke_score < self.kinetic_threshold:
            return 0

        # ---------- 2) SIGNED ORDERFLOW ----------
        price_short = prices[-self.vol_window:]
        dp = np.diff(price_short)
        v = trade_vol[-len(dp):]

        signed_vol = np.sum(np.sign(dp) * v)
        total_trade_vol = v.sum() + 1e-9
        signed_vol_ratio = signed_vol / total_trade_vol  # -1..+1

        # ---------- 3) SHORT & MEDIUM SLOPES ----------
        fast_len = min(self.fast_lookback, len(prices))
        slow_len = min(self.slow_lookback, len(prices))

        fast_slice = prices[-fast_len:]
        slow_slice = prices[-slow_len:]

        fast_slope = self._lin_slope(fast_slice)
        slow_slope = self._lin_slope(slow_slice)

        price_scale = np.mean(prices[-slow_len:]) + 1e-9
        fast_slope_norm = fast_slope / price_scale
        slow_slope_norm = slow_slope / price_scale

        # ---------- 4) LOCAL RANGE POSITION ----------
        window_slice = prices[-self.price_window:]
        p_min = window_slice.min()
        p_max = window_slice.max()
        rng = (p_max - p_min) + 1e-9
        range_pos = (window_slice[-1] - p_min) / rng  # 0 = local low, 1 = local high

        # ---------- 5) VOTING ----------
        long_votes = 0
        short_votes = 0

        # (a) Signed orderflow bias
        if signed_vol_ratio > 0.15:
            long_votes += 1
        elif signed_vol_ratio < -0.15:
            short_votes += 1

        # (b) Fast slope
        if fast_slope_norm > 0:
            long_votes += 1
        elif fast_slope_norm < 0:
            short_votes += 1

        # (c) Slow slope
        if slow_slope_norm > 0:
            long_votes += 1
        elif slow_slope_norm < 0:
            short_votes += 1

        # (d) Range position
        if range_pos > 0.65:
            long_votes += 1
        elif range_pos < 0.35:
            short_votes += 1

        # ---------- 6) FINAL DECISION ----------
        max_votes = max(long_votes, short_votes)
        if max_votes < self.min_votes:
            return 0

        if long_votes >= short_votes + 1:
            return +1
        elif short_votes >= long_votes + 1:
            return -1
        else:
            return 0


# ==========================================
# DATA LOADING
# ==========================================
def download_futures_for_day(year, month, day, symbol, fut_ts, day_folder):
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{symbol}/{fut_ts}.parquet"
    local_path = os.path.join(day_folder, "FUT.parquet")
    ok = download_parquet_to_path(key, local_path)
    if not ok:
        return None

    try:
        df = pd.read_parquet(local_path)
    except Exception:
        return None

    if "DateTime" not in df.columns:
        if "Date" in df.columns and "Time" in df.columns:
            df["DateTime"] = pd.to_datetime(
                df["Date"].astype(str) + " " + df["Time"].astype(str),
                dayfirst=True,
                errors="coerce"
            )
        else:
            return None

    df = df.dropna(subset=["DateTime"])
    df["LTP"] = pd.to_numeric(df.get("LTP"), errors="coerce")
    df = df.dropna(subset=["LTP"])

    if "Volume" in df.columns:
        df["Volume"] = pd.to_numeric(df["Volume"], errors="coerce")
    elif "OpenInterest" in df.columns:
        df["Volume"] = pd.to_numeric(df["OpenInterest"], errors="coerce")
    else:
        df["Volume"] = 0.0

    df = df.sort_values("DateTime").reset_index(drop=True)

    # Restrict to normal trading hours (optional)
    df = df[
        (df["DateTime"].dt.time >= datetime.time(9, 15)) &
        (df["DateTime"].dt.time <= datetime.time(15, 15))
    ].reset_index(drop=True)

    return df


# ==========================================
# DAY PROCESSING
# ==========================================
def process_day_trades(df):
    brain = KineticDirectionalBrain(
        kinetic_threshold=KINETIC_THRESHOLD,
        price_window=PRICE_WINDOW,
        vol_window=VOL_WINDOW,
        fast_lookback=FAST_LOOKBACK,
        slow_lookback=SLOW_LOOKBACK,
        min_votes=MIN_VOTES
    )

    trades = []
    in_trade = False
    entry_time = None
    entry_price = 0.0
    side = None
    daily_pnl = 0.0

    for _, row in df.iterrows():
        ts = row["DateTime"]
        ltp = float(row["LTP"])
        vol = float(row["Volume"])

        # Hard daily stop in INR
        if daily_pnl <= DAILY_MAX_LOSS:
            break

        # ----- Manage open trade -----
        if in_trade:
            duration = (ts - entry_time).total_seconds()
            exit_signal = False
            exit_reason = ""
            pnl_pts = 0.0

            if side == "LONG":
                pnl_pts = ltp - entry_price
                if pnl_pts <= -STOP_LOSS_POINTS:
                    exit_signal = True
                    exit_reason = "SL"
                elif pnl_pts >= TAKE_PROFIT_POINTS:
                    exit_signal = True
                    exit_reason = "TP"
            else:  # SHORT
                pnl_pts = entry_price - ltp
                if pnl_pts <= -STOP_LOSS_POINTS:
                    exit_signal = True
                    exit_reason = "SL"
                elif pnl_pts >= TAKE_PROFIT_POINTS:
                    exit_signal = True
                    exit_reason = "TP"

            if not exit_signal and duration >= MAX_HOLD_SECONDS:
                exit_signal = True
                exit_reason = "TIME"

            if exit_signal:
                gross_pnl_inr = pnl_pts * LOT_SIZE
                cost_inr = FUTURES_ROUND_TRIP_COST * LOT_SIZE
                net_inr = gross_pnl_inr - cost_inr
                net_pts_after_cost = pnl_pts - FUTURES_ROUND_TRIP_COST
                daily_pnl += net_inr

                trades.append({
                    "Date": ts.date(),
                    "Entry_Time": entry_time,
                    "Exit_Time": ts,
                    "Side": side,
                    "Entry_Price": entry_price,
                    "Exit_Price": ltp,
                    "Raw_PnL_Pts": pnl_pts,
                    "Net_Pts": net_pts_after_cost,
                    "Net_INR": net_inr,
                    "Exit_Reason": exit_reason,
                })

                in_trade = False
                side = None
                entry_time = None
                entry_price = 0.0
                continue

        # ----- Generate new signal when flat -----
        if not in_trade:
            sig = brain.tick(ltp, vol)
            if sig == 1:
                in_trade = True
                side = "LONG"
                entry_time = ts
                entry_price = ltp
            elif sig == -1:
                in_trade = True
                side = "SHORT"
                entry_time = ts
                entry_price = ltp
        else:
            # Still call brain to keep buffers moving
            brain.tick(ltp, vol)

    return pd.DataFrame(trades)


# ==========================================
# MAIN
# ==========================================
def main():
    all_trades = []

    for day in range(1, 31):
        day_folder = f"data/{YEAR}-{MONTH:02d}-{day:02d}/"
        os.makedirs(day_folder, exist_ok=True)

        print(f"{day:02d}/{MONTH}/{YEAR}...", end=" ")

        df = download_futures_for_day(YEAR, MONTH, day, SYMBOL, FUT_TS, day_folder)
        if df is None or df.empty:
            print("No data")
            continue

        trades_df = process_day_trades(df)

        if trades_df.empty:
            print("No trades")
            continue

        day_pnl = trades_df["Net_INR"].sum()
        n_trades = len(trades_df)
        sl_hits = (trades_df["Exit_Reason"] == "SL").sum()
        tp_hits = (trades_df["Exit_Reason"] == "TP").sum()

        print(f"Trades: {n_trades} (SL:{sl_hits}/TP:{tp_hits}) | PnL: ₹{day_pnl:,.0f}")
        if day_pnl <= DAILY_MAX_LOSS:
            print("   [RISK] Daily Stop Loss Breached!")

        daily_csv = os.path.join(day_folder, "futures_directional_results.csv")
        trades_df.to_csv(daily_csv, index=False)
        all_trades.append(trades_df)

    if not all_trades:
        print("\nNo trades for the month.")
        return

    full_df = pd.concat(all_trades, ignore_index=True)

    total_pnl = full_df["Net_INR"].sum()
    win_rate = (full_df["Net_Pts"] > 0).mean() * 100
    longs = full_df[full_df["Side"] == "LONG"]
    shorts = full_df[full_df["Side"] == "SHORT"]

    print("\n" + "="*60)
    print(f"KINETIC DIRECTIONAL SUMMARY: {MONTH}/{YEAR}")
    print("="*60)
    print(f"Total Net PnL:      ₹{total_pnl:,.2f}")
    print(f"Win Rate:           {win_rate:.1f}%")
    print(f"Total Trades:       {len(full_df)}")
    print(f"  - Longs:          {len(longs)} (₹{longs['Net_INR'].sum():,.0f})")
    print(f"  - Shorts:         {len(shorts)} (₹{shorts['Net_INR'].sum():,.0f})")

    out_csv = f"data/nov_{YEAR}_kinetic_directional_results.csv"
    full_df.to_csv(out_csv, index=False)
    print(f"\nResults saved: {out_csv}")


if __name__ == "__main__":
    main()



KINETIC DIRECTIONAL FUTURES SCALPER - 11/2025
Symbol: NIFTY, Contract: NIFTY25NOVFUT

01/11/2025... No data
02/11/2025... No data
03/11/2025... No data
04/11/2025... Trades: 2 (SL:2/TP:0) | PnL: ₹-4,035
   [RISK] Daily Stop Loss Breached!
05/11/2025... No data
06/11/2025... Trades: 23 (SL:4/TP:3) | PnL: ₹16,770
07/11/2025... Trades: 2 (SL:2/TP:0) | PnL: ₹-3,900
   [RISK] Daily Stop Loss Breached!
08/11/2025... No data
09/11/2025... No data
10/11/2025... No data
11/11/2025... Trades: 2 (SL:2/TP:0) | PnL: ₹-4,418
   [RISK] Daily Stop Loss Breached!
12/11/2025... Trades: 24 (SL:5/TP:1) | PnL: ₹-2,798
13/11/2025... Trades: 2 (SL:2/TP:0) | PnL: ₹-4,050
   [RISK] Daily Stop Loss Breached!
14/11/2025... No data
15/11/2025... No data
16/11/2025... No data
17/11/2025... Trades: 23 (SL:2/TP:1) | PnL: ₹-532
18/11/2025... Trades: 23 (SL:4/TP:1) | PnL: ₹-878
19/11/2025... Trades: 24 (SL:4/TP:0) | PnL: ₹-263
20/11/2025... Trades: 16 (SL:3/TP:0) | PnL: ₹-4,282
   [RISK] Daily Stop Loss Breached!
21/

In [ ]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

# Expiry Dates 2025
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# Strategy Params (From your snippet)
KINETIC_THRESHOLD = 37500
HOLD_SECONDS = 900
COST_PER_TRADE = 1.0     
LOT_SIZE = 75

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        
        # Handle Volume
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. ORIGINAL KINETIC BRAIN
# ==========================================
class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=900):
        self.threshold = threshold
        self.hold_seconds = hold_seconds
        self.tick_buffer = deque(maxlen=50) 
        self.in_trade = False
        self.entry_time = 0
        self.last_score = 0

    # Adapted to accept 'timestamp' argument for backtesting
    def process_tick(self, ltp, cumulative_volume, timestamp):
        # Convert timestamp to float seconds for calculation
        current_time = timestamp.timestamp()
        
        self.tick_buffer.append([ltp, cumulative_volume])
        
        if len(self.tick_buffer) < 50:
            return 0
            
        if self.in_trade:
            time_passed = current_time - self.entry_time
            if time_passed >= self.hold_seconds:
                self.in_trade = False
                self.entry_time = 0
                return -1 # exit
            else:
                return 0 # hold
                
        score = self._calculate_score()
        self.last_score = score
        
        if score > self.threshold:
            self.in_trade = True
            self.entry_time = current_time
            return 1 # enter
            
        return 0 # wait

    def _calculate_score(self):
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        vol_diff = np.diff(vols)
        # Ensure positive volume flow (filter resets)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        
        displacement = abs(prices[-1] - prices[0])
        total_vol = np.sum(trade_vol)
        
        score = total_vol / (displacement + 0.05)
        return score

# ==========================================
# 3. DAILY BACKTEST ENGINE
# ==========================================
def run_day(df):
    brain = KineticBrain(threshold=KINETIC_THRESHOLD, hold_seconds=HOLD_SECONDS)
    
    trades = []
    active_trade_price = 0.0
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        sig = brain.process_tick(ltp, vol, ts)
        
        if sig == 1:
            # ENTRY (Long)
            active_trade_price = ltp
            
        elif sig == -1:
            # EXIT
            pnl = ltp - active_trade_price
            trades.append(pnl - COST_PER_TRADE)
            active_trade_price = 0.0
            
    return trades

# ==========================================
# 4. MAIN LOOP
# ==========================================
def main():
    print(f"Running ORIGINAL KINETIC from {START_DATE} to {END_DATE}")
    print(f"Threshold: {KINETIC_THRESHOLD} | Hold: {HOLD_SECONDS}s | Cost: {COST_PER_TRADE}")
    print("-" * 65)
    print(f"{'Date':<12} | {'Trades':<6} | {'Daily PnL (Pts)':<15} | {'Status'}")
    print("-" * 65)
    
    current_date = START_DATE
    total_points = 0.0
    total_trades = 0
    date_pnl_map = []
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            daily_pts = sum(daily_trades)
            count = len(daily_trades)
            
            total_points += daily_pts
            total_trades += count
            
            status = "PROFIT" if daily_pts > 0 else "LOSS" if daily_pts < 0 else "FLAT"
            print(f"{current_date} | {count:<6} | {daily_pts:>10.2f} pts   | {status}")
            
            date_pnl_map.append({'Date': current_date, 'PnL': daily_pts})
        
        current_date += datetime.timedelta(days=1)
        
    print("-" * 65)
    print(f"TOTAL RESULTS:")
    print(f"Gross Points: {total_points:.2f}")
    print(f"Total Trades: {total_trades}")
    print(f"Avg Pts/Day:  {total_points / len(date_pnl_map) if date_pnl_map else 0:.2f}")
    print(f"Total INR:    ₹{total_points * LOT_SIZE:,.2f}")

if __name__ == "__main__":
    main()

Running ORIGINAL KINETIC from 2025-09-01 to 2025-11-26
Threshold: 37500 | Hold: 900s | Cost: 1.0
-----------------------------------------------------------------
Date         | Trades | Daily PnL (Pts) | Status
-----------------------------------------------------------------
2025-09-01 | 23     |      80.50 pts   | PROFIT
2025-09-02 | 22     |    -109.80 pts   | LOSS
2025-09-03 | 17     |      85.70 pts   | PROFIT
2025-09-04 | 23     |    -176.80 pts   | LOSS
2025-09-05 | 23     |     -74.20 pts   | LOSS
2025-09-08 | 22     |     -90.50 pts   | LOSS
2025-09-09 | 20     |     -26.30 pts   | LOSS
2025-09-10 | 23     |      41.00 pts   | PROFIT
2025-09-11 | 20     |      59.70 pts   | PROFIT
2025-09-12 | 24     |      31.10 pts   | PROFIT
2025-09-18 | 23     |      -0.50 pts   | LOSS
2025-09-22 | 22     |     -40.40 pts   | LOSS
2025-09-23 | 23     |     -79.00 pts   | LOSS
2025-09-25 | 22     |    -108.70 pts   | LOSS
2025-10-03 | 13     |      74.80 pts   | PROFIT
2025-10-06 | 23     

In [ ]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

# Expiry Dates 2025
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# Strategy Params (Original)
KINETIC_THRESHOLD = 37500
HOLD_SECONDS = 900
COST_PER_TRADE = 1.0     
LOT_SIZE = 75

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        
        # Handle Volume
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. DIRECTIONAL KINETIC BRAIN
# ==========================================
class KineticBrain:
    def __init__(self, threshold=37500, hold_seconds=900):
        self.threshold = threshold
        self.hold_seconds = hold_seconds
        self.tick_buffer = deque(maxlen=50) 
        self.in_trade = False
        self.entry_time = 0
        self.last_score = 0

    def process_tick(self, ltp, cumulative_volume, timestamp):
        current_time = timestamp.timestamp()
        
        self.tick_buffer.append([ltp, cumulative_volume])
        
        if len(self.tick_buffer) < 50:
            return 0
            
        if self.in_trade:
            time_passed = current_time - self.entry_time
            if time_passed >= self.hold_seconds:
                self.in_trade = False
                self.entry_time = 0
                return -1 # exit signal
            else:
                return 0 # hold
                
        score = self._calculate_score()
        self.last_score = score
        
        if score > self.threshold:
            # FIX: Determine direction from price drift in buffer
            # Even if displacement is small, the sign tells us the pressure
            data = np.array(self.tick_buffer)
            price_change = data[-1, 0] - data[0, 0]
            
            self.in_trade = True
            self.entry_time = current_time
            
            if price_change >= 0:
                return 1 # LONG
            else:
                return 2 # SHORT
            
        return 0 

    def _calculate_score(self):
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        
        displacement = abs(prices[-1] - prices[0])
        total_vol = np.sum(trade_vol)
        
        score = total_vol / (displacement + 0.05)
        return score

# ==========================================
# 3. DAILY BACKTEST ENGINE
# ==========================================
def run_day(df):
    brain = KineticBrain(threshold=KINETIC_THRESHOLD, hold_seconds=HOLD_SECONDS)
    
    trades = []
    active_trade_price = 0.0
    direction = 0 # 1 Long, -1 Short
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        sig = brain.process_tick(ltp, vol, ts)
        
        if sig == 1: # LONG
            active_trade_price = ltp
            direction = 1
            
        elif sig == 2: # SHORT
            active_trade_price = ltp
            direction = -1
            
        elif sig == -1: # EXIT
            pnl = 0
            if direction == 1:
                pnl = ltp - active_trade_price
            elif direction == -1:
                pnl = active_trade_price - ltp
                
            trades.append(pnl - COST_PER_TRADE)
            active_trade_price = 0.0
            direction = 0
            
    return trades

# ==========================================
# 4. MAIN LOOP
# ==========================================
def main():
    print(f"Running DIRECTIONAL KINETIC from {START_DATE} to {END_DATE}")
    print(f"Threshold: {KINETIC_THRESHOLD} | Hold: {HOLD_SECONDS}s | Cost: {COST_PER_TRADE}")
    print("-" * 65)
    print(f"{'Date':<12} | {'Trades':<6} | {'Daily PnL (Pts)':<15} | {'Status'}")
    print("-" * 65)
    
    current_date = START_DATE
    total_points = 0.0
    total_trades = 0
    date_pnl_map = []
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            daily_pts = sum(daily_trades)
            count = len(daily_trades)
            
            total_points += daily_pts
            total_trades += count
            
            status = "PROFIT" if daily_pts > 0 else "LOSS" if daily_pts < 0 else "FLAT"
            print(f"{current_date} | {count:<6} | {daily_pts:>10.2f} pts   | {status}")
            
            date_pnl_map.append({'Date': current_date, 'PnL': daily_pts})
        
        current_date += datetime.timedelta(days=1)
        
    print("-" * 65)
    print(f"TOTAL RESULTS:")
    print(f"Gross Points: {total_points:.2f}")
    print(f"Total Trades: {total_trades}")
    print(f"Avg Pts/Day:  {total_points / len(date_pnl_map) if date_pnl_map else 0:.2f}")
    print(f"Total INR:    ₹{total_points * LOT_SIZE:,.2f}")

if __name__ == "__main__":
    main()

Running DIRECTIONAL KINETIC from 2025-09-01 to 2025-11-26
Threshold: 37500 | Hold: 900s | Cost: 1.0
-----------------------------------------------------------------
Date         | Trades | Daily PnL (Pts) | Status
-----------------------------------------------------------------
2025-09-01 | 23     |      24.90 pts   | PROFIT
2025-09-02 | 22     |     -46.80 pts   | LOSS
2025-09-03 | 17     |      65.70 pts   | PROFIT
2025-09-04 | 23     |    -162.20 pts   | LOSS
2025-09-05 | 23     |     -79.00 pts   | LOSS
2025-09-08 | 22     |    -108.90 pts   | LOSS
2025-09-09 | 20     |     -54.70 pts   | LOSS
2025-09-10 | 23     |      41.00 pts   | PROFIT
2025-09-11 | 20     |      59.70 pts   | PROFIT
2025-09-12 | 24     |      14.50 pts   | PROFIT
2025-09-18 | 23     |     -11.50 pts   | LOSS
2025-09-22 | 22     |    -114.40 pts   | LOSS
2025-09-23 | 23     |     199.80 pts   | PROFIT
2025-09-25 | 22     |     -21.50 pts   | LOSS
2025-10-03 | 13     |      54.80 pts   | PROFIT
2025-10-06 | 23

In [50]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# Strategy Params
KINETIC_THRESHOLD = 37500
MEASURE_DURATION = 900   # 15 Minutes to measure the "Bang"

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. KINETIC BRAIN (MAGNITUDE ONLY)
# ==========================================
class KineticBrain:
    def __init__(self, threshold=37500):
        self.threshold = threshold
        self.tick_buffer = deque(maxlen=50) 
        self.last_score = 0

    def process_tick(self, ltp, cumulative_volume):
        self.tick_buffer.append([ltp, cumulative_volume])
        
        if len(self.tick_buffer) < 50:
            return False # Not ready
            
        score = self._calculate_score()
        self.last_score = score
        
        if score > self.threshold:
            return True # SIGNAL FIRED (High Energy)
            
        return False

    def _calculate_score(self):
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        
        displacement = abs(prices[-1] - prices[0])
        total_vol = np.sum(trade_vol)
        
        score = total_vol / (displacement + 0.05)
        return score

# ==========================================
# 3. MAGNITUDE TEST ENGINE
# ==========================================
def run_day(df):
    brain = KineticBrain(threshold=KINETIC_THRESHOLD)
    
    events = []
    
    # State
    monitoring = False
    entry_time = None
    entry_price = 0.0
    max_dev = 0.0 # Tracks max absolute move
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        # 1. Update Monitor if active
        if monitoring:
            elapsed = (ts - entry_time).total_seconds()
            current_dev = abs(ltp - entry_price)
            
            if current_dev > max_dev:
                max_dev = current_dev
                
            if elapsed >= MEASURE_DURATION:
                # Event Finished
                events.append(max_dev)
                monitoring = False
                brain.tick_buffer.clear() # Reset brain to find fresh energy
            continue # Skip detection while monitoring an event
            
        # 2. Check Signal
        fired = brain.process_tick(ltp, vol)
        
        if fired:
            monitoring = True
            entry_time = ts
            entry_price = ltp
            max_dev = 0.0 # Reset tracker
            
    return events

# ==========================================
# 4. MAIN LOOP
# ==========================================
def main():
    print(f"Running KINETIC MAGNITUDE TEST from {START_DATE} to {END_DATE}")
    print(f"Threshold: {KINETIC_THRESHOLD} | Measure Window: {MEASURE_DURATION}s")
    print("-" * 65)
    
    all_moves = []
    
    current_date = START_DATE
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_moves = run_day(df)
            all_moves.extend(daily_moves)
            
            # Optional: Print daily avg just to show progress
            avg = sum(daily_moves)/len(daily_moves) if daily_moves else 0
            print(f"{current_date} | {len(daily_moves)} Signals | Avg Move: {avg:.2f} pts")
        
        current_date += datetime.timedelta(days=1)
        
    # --- FINAL STATS ---
    total = len(all_moves)
    if total == 0:
        print("No signals found.")
        return

    moves = np.array(all_moves)
    
    print("\n" + "="*40)
    print(f"MAGNITUDE DISTRIBUTION (N={total})")
    print("="*40)
    
    thresholds = [10, 20, 30, 40, 50, 75, 100]
    
    for t in thresholds:
        count = np.sum(moves >= t)
        pct = (count / total) * 100
        print(f"Moves >= {t:<3} pts: {count:<5} ({pct:.1f}%)")
        
    print("-" * 40)
    print(f"Average Move: {np.mean(moves):.2f} pts")
    print(f"Median Move:  {np.median(moves):.2f} pts")
    print(f"Max Move:     {np.max(moves):.2f} pts")

if __name__ == "__main__":
    main()

Running KINETIC MAGNITUDE TEST from 2025-09-01 to 2025-11-26
Threshold: 37500 | Measure Window: 900s
-----------------------------------------------------------------
2025-09-01 | 22 Signals | Avg Move: 17.59 pts
2025-09-02 | 22 Signals | Avg Move: 29.28 pts
2025-09-03 | 17 Signals | Avg Move: 27.08 pts
2025-09-04 | 22 Signals | Avg Move: 31.04 pts
2025-09-05 | 22 Signals | Avg Move: 36.62 pts
2025-09-08 | 22 Signals | Avg Move: 28.82 pts
2025-09-09 | 19 Signals | Avg Move: 18.77 pts
2025-09-10 | 23 Signals | Avg Move: 20.67 pts
2025-09-11 | 18 Signals | Avg Move: 16.32 pts
2025-09-12 | 23 Signals | Avg Move: 12.48 pts
2025-09-18 | 22 Signals | Avg Move: 17.09 pts
2025-09-22 | 20 Signals | Avg Move: 21.57 pts
2025-09-23 | 21 Signals | Avg Move: 26.79 pts
2025-09-25 | 22 Signals | Avg Move: 27.22 pts
2025-10-03 | 13 Signals | Avg Move: 22.11 pts
2025-10-06 | 22 Signals | Avg Move: 21.42 pts
2025-10-07 | 22 Signals | Avg Move: 25.55 pts
2025-10-08 | 22 Signals | Avg Move: 32.19 pts
2025-

In [51]:
df = pd.read_csv('master_fut_df.csv')
df

,Date,Time,Trading_Symbol,Instrument_Token,LTP,LTQ,Volume,Open_Interest,BestBid,BestAsk,BidSize,AskSize,Ticker
0,01/09/2025,08:46:42.278,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
1,01/09/2025,09:10:01.994,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
2,01/09/2025,09:15:01.722,NIFTY25SEPFUT,13568258,24590.0,300,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
3,01/09/2025,09:15:02.471,NIFTY25SEPFUT,13568258,24602.8,75,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
4,01/09/2025,09:15:02.975,NIFTY25SEPFUT,13568258,24595.7,75,5700,16610100,24595.7,24605.5,300,300,NIFTY25SEPFUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213543,13/11/2025,15:29:33.077,NIFTY25NOVFUT,9485826,25957.3,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213544,13/11/2025,15:29:34.659,NIFTY25NOVFUT,9485826,25957.5,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213545,13/11/2025,15:29:35.142,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT
1213546,13/11/2025,15:29:35.834,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT


In [ ]:
import pandas as pd
import numpy as np
from collections import deque
import time
import os
import datetime

# ==========================================
# CONFIGURATION
# ==========================================
KINETIC_THRESHOLD = 
HOLD_SECONDS = 900  # 15 Minutes hold logic

# Trading Hours
MARKET_START = datetime.time(9, 15)
MARKET_END = datetime.time(15, 30)

# ==========================================
# 1. THE PRODUCTION CLASS (Copy this to your live bot)
# ==========================================
class LiveKineticBrain:
    def __init__(self, threshold=37500):
        """
        Initialize the Kinetic Brain.
        :param threshold: The energy score required to trigger a signal.
        """
        self.threshold = threshold
        # Buffer stores [price, cumulative_volume]
        self.tick_buffer = deque(maxlen=50) 
        self.last_score = 0.0
        self.is_ready = False

    def process_tick(self, ltp, cumulative_volume):
        """
        Feed a single tick into the brain.
        :param ltp: Last Traded Price (float)
        :param cumulative_volume: Total Volume / LTQ Sum / Open Interest (float)
        :return: True if Signal Fired, False otherwise
        """
        # 1. Store Tick
        self.tick_buffer.append([ltp, cumulative_volume])
        
        # 2. Warmup Check
        if len(self.tick_buffer) < 50:
            self.is_ready = False
            return False 
            
        self.is_ready = True

        # 3. Calculate Score
        score = self._calculate_score()
        self.last_score = score
        
        # 4. Check Trigger
        if score > self.threshold:
            return True # SIGNAL FIRED!
            
        return False

    def _calculate_score(self):
        """Internal calculation logic (Vectorized for speed)"""
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        # Calculate Flow (Positive volume deltas only)
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        
        # Calculate Displacement (Price change over 50 ticks)
        displacement = abs(prices[-1] - prices[0])
        
        # Calculate Energy Score
        total_vol = np.sum(trade_vol)
        score = total_vol / (displacement + 0.05)
        
        return score

# ==========================================
# 2. LIVE SIMULATION (MOCK WEBSOCKET)
# ==========================================
def simulate_live_market(csv_file_path):
    print(f"--- STARTING LIVE SIMULATION ON {csv_file_path} ---")
    
    # Load Data
    try:
        print("Loading CSV (This may take a moment for large files)...")
        df = pd.read_csv(csv_file_path)
        
        # --- ROBUST DATE PARSING ---
        # Combine Date and Time if they are separate
        if 'Date' in df.columns and 'Time' in df.columns:
            # Format: 01/09/2025 + 08:46:42.278
            print("Parsing DateTime from 'Date' and 'Time' columns...")
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + ' ' + df['Time'].astype(str), 
                format='%d/%m/%Y %H:%M:%S.%f', 
                errors='coerce'
            )
            # Fallback for rows without millis or different formats if coerce fails
            mask_nan = df['DateTime'].isna()
            if mask_nan.any():
                 df.loc[mask_nan, 'DateTime'] = pd.to_datetime(
                    df.loc[mask_nan, 'Date'].astype(str) + ' ' + df.loc[mask_nan, 'Time'].astype(str), 
                    errors='coerce' # Just drop them if still failing
                )
        elif 'DateTime' in df.columns:
             df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True)
        
        # Drop bad rows
        df = df.dropna(subset=['DateTime']).sort_values('DateTime').reset_index(drop=True)
        print(f"Loaded {len(df)} valid ticks. Starting feed...")
        
    except FileNotFoundError:
        print("Error: File not found.")
        return None

    # Initialize the Brain
    brain = LiveKineticBrain(threshold=KINETIC_THRESHOLD)
    
    trades = []
    in_trade = False
    entry_data = {} 
    
    # --- THE EVENT LOOP ---
    for i, row in df.iterrows():
        # Extract fields (Handle aliases)
        ltp = row.get('LTP') or row.get('LastTradedPrice') or row.get('Close')
        vol = row.get('Volume') or row.get('LTQ') or row.get('OpenInterest')
        ts = row['DateTime']
        
        if pd.isna(ltp) or pd.isna(vol): continue
            
        curr_time = ts.time()
        
        # 1. FEED THE BRAIN (Always feed to keep buffer current)
        signal_fired = brain.process_tick(ltp, vol)
        
        # 2. TRADE MANAGEMENT
        if in_trade:
            time_in_trade = (ts - entry_data['Entry_Time']).total_seconds()
            should_exit = False
            exit_reason = ""
            
            # Condition A: Time Limit
            if time_in_trade >= HOLD_SECONDS:
                should_exit = True
                exit_reason = "TIME_LIMIT"
            
            # Condition B: Market Close Force Exit (15:30)
            if curr_time >= MARKET_END:
                should_exit = True
                exit_reason = "MARKET_CLOSE"
            
            if should_exit:
                exit_price = ltp
                pnl = exit_price - entry_data['Entry_Price'] # Long Logic
                
                trades.append({
                    'Entry_Time': entry_data['Entry_Time'],
                    'Entry_Price': entry_data['Entry_Price'],
                    'Exit_Time': ts,
                    'Exit_Price': exit_price,
                    'Score': entry_data['Score'],
                    'PnL_Points': pnl,
                    'Reason': exit_reason
                })
                
                # print(f"🔴 CLOSED ({exit_reason}) at {ts.time()} | PnL: {pnl:.2f}")
                in_trade = False
                entry_data = {}

        # 3. ENTRY LOGIC
        else:
            # Only enter if within Market Hours (09:15 - 15:30)
            if curr_time >= MARKET_START and curr_time < MARKET_END:
                if signal_fired:
                    in_trade = True
                    entry_data = {
                        'Entry_Time': ts,
                        'Entry_Price': ltp,
                        'Score': brain.last_score
                    }
                    # print(f"🟢 OPEN LONG at {ts.time()} | Price: {ltp} | Score: {brain.last_score:.0f}")
    
    # Force close if still open at end of file
    if in_trade:
         trades.append({
            'Entry_Time': entry_data['Entry_Time'],
            'Entry_Price': entry_data['Entry_Price'],
            'Exit_Time': df.iloc[-1]['DateTime'],
            'Exit_Price': df.iloc[-1]['LTP'], # Close at last tick
            'Score': entry_data['Score'],
            'PnL_Points': df.iloc[-1]['LTP'] - entry_data['Entry_Price'],
            'Reason': "END_OF_DATA"
        })

    print(f"--- SIMULATION ENDED. Total Trades: {len(trades)} ---")
    
    # Results
    if trades:
        trade_df = pd.DataFrame(trades)
        
        total_pnl = trade_df['PnL_Points'].sum()
        win_rate = (trade_df['PnL_Points'] > 0).mean() * 100
        
        print("\n=== RESULTS SUMMARY ===")
        print(f"Total PnL (Points): {total_pnl:.2f}")
        print(f"Total PnL (INR):    ₹{total_pnl * 75:,.2f}")
        print(f"Win Rate:           {win_rate:.2f}%")
        print(f"Avg PnL per Trade:  {trade_df['PnL_Points'].mean():.2f}")
        
        print("\n=== FIRST 5 TRADES ===")
        print(trade_df.head())
        return trade_df
    else:
        print("No trades generated.")
        return pd.DataFrame()

if __name__ == "__main__":
    # Use 'master_fut_df.csv' if available, otherwise generate dummy
    FILENAME = "master_fut_df.csv"
    
    if not os.path.exists(FILENAME):
        print(f"'{FILENAME}' not found. Creating dummy data for demo...")
        dummy_data = {
            'Date': ['01/09/2025'] * 3000,
            'Time': [pd.Timestamp(f'2025-09-01 09:15:00') + pd.Timedelta(seconds=i) for i in range(3000)],
        }
        # Convert timestamps back to string format to match user requirement
        dummy_data['Time'] = [t.strftime('%H:%M:%S.%f')[:-3] for t in dummy_data['Time']]
        
        dummy_df = pd.DataFrame(dummy_data)
        dummy_df['LTP'] = np.random.normal(24000, 5, 3000)
        dummy_df['Volume'] = np.cumsum(np.random.randint(100, 5000, 3000))
        
        # Spike
        dummy_df.loc[100:110, 'Volume'] += 200000
        
        dummy_df.to_csv(FILENAME, index=False)

    df_results = simulate_live_market(FILENAME)

--- STARTING LIVE SIMULATION ON master_fut_df.csv ---
Loading CSV (This may take a moment for large files)...
Parsing DateTime from 'Date' and 'Time' columns...
Loaded 1213548 valid ticks. Starting feed...
--- SIMULATION ENDED. Total Trades: 853 ---

=== RESULTS SUMMARY ===
Total PnL (Points): 622.60
Total PnL (INR):    ₹46,695.00
Win Rate:           51.70%
Avg PnL per Trade:  0.73

=== FIRST 5 TRADES ===
               Entry_Time  Entry_Price               Exit_Time  Exit_Price  \
0 2025-09-01 09:16:04.406      24637.3 2025-09-01 09:31:04.506     24665.1   
1 2025-09-01 09:31:36.229      24673.4 2025-09-01 09:46:36.292     24677.3   
2 2025-09-01 09:46:37.146      24680.0 2025-09-01 10:01:38.278     24680.0   
3 2025-09-01 10:01:44.319      24680.0 2025-09-01 10:16:44.536     24659.5   
4 2025-09-01 10:16:54.795      24660.0 2025-09-01 10:31:55.034     24652.0   

      Score  PnL_Points      Reason  
0   45300.0        27.8  TIME_LIMIT  
1  184500.0         3.9  TIME_LIMIT  
2  20400

In [53]:
trade_df

NameError: name 'trade_df' is not defined

In [54]:
df_results

,Entry_Time,Entry_Price,Exit_Time,Exit_Price,Score,PnL_Points,Reason
0,2025-09-01 09:16:04.406,24637.3,2025-09-01 09:31:04.506,24665.1,45300.000000,27.8,TIME_LIMIT
1,2025-09-01 09:31:36.229,24673.4,2025-09-01 09:46:36.292,24677.3,184500.000000,3.9,TIME_LIMIT
2,2025-09-01 09:46:37.146,24680.0,2025-09-01 10:01:38.278,24680.0,204000.000000,0.0,TIME_LIMIT
3,2025-09-01 10:01:44.319,24680.0,2025-09-01 10:16:44.536,24659.5,121500.000000,-20.5,TIME_LIMIT
4,2025-09-01 10:16:54.795,24660.0,2025-09-01 10:31:55.034,24652.0,46500.000000,-8.0,TIME_LIMIT
...,...,...,...,...,...,...,...
848,2025-11-13 14:19:43.656,25986.0,2025-11-13 14:34:45.960,25986.8,130500.000000,0.8,TIME_LIMIT
849,2025-11-13 14:36:57.217,25979.0,2025-11-13 14:51:57.613,25975.0,117000.000000,-4.0,TIME_LIMIT
850,2025-11-13 14:52:39.178,25970.0,2025-11-13 15:07:39.839,25953.0,69000.000000,-17.0,TIME_LIMIT
851,2025-11-13 15:07:46.298,25950.0,2025-11-13 15:22:48.527,25958.8,71500.000001,8.8,TIME_LIMIT


In [55]:
df_results['PnL_Points'].sum()

622.599999999984

In [58]:
import pandas as pd
import numpy as np
from collections import deque
import time
import os
import datetime
import sys

# ==========================================
# CONFIGURATION
# ==========================================
KINETIC_THRESHOLD = 37500
HOLD_SECONDS = 900  # 15 Minutes hold logic

# Trading Hours
MARKET_START = datetime.time(9, 15)
MARKET_END = datetime.time(15, 30)

# ==========================================
# 1. THE PRODUCTION CLASS (Copy this to your live bot)
# ==========================================
class LiveKineticBrain:
    def __init__(self, threshold=37500):
        """
        Initialize the Kinetic Brain.
        :param threshold: The energy score required to trigger a signal.
        """
        self.threshold = threshold
        # Buffer stores [price, cumulative_volume]
        self.tick_buffer = deque(maxlen=50) 
        self.last_score = 0.0
        self.is_ready = False

    def process_tick(self, ltp, cumulative_volume):
        """
        Feed a single tick into the brain.
        :param ltp: Last Traded Price (float)
        :param cumulative_volume: Total Volume / LTQ Sum / Open Interest (float)
        :return: True if Signal Fired, False otherwise
        """
        # 1. Store Tick
        self.tick_buffer.append([ltp, cumulative_volume])
        
        # 2. Warmup Check
        if len(self.tick_buffer) < 50:
            self.is_ready = False
            return False 
            
        self.is_ready = True

        # 3. Calculate Score
        score = self._calculate_score()
        self.last_score = score
        
        # 4. Check Trigger
        if score > self.threshold:
            return True # SIGNAL FIRED!
            
        return False

    def _calculate_score(self):
        """Internal calculation logic (Vectorized for speed)"""
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        # Calculate Flow (Positive volume deltas only)
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        
        # Calculate Displacement (Price change over 50 ticks)
        displacement = abs(prices[-1] - prices[0])
        
        # Calculate Energy Score
        total_vol = np.sum(trade_vol)
        score = total_vol / (displacement + 0.05)
        
        return score

# ==========================================
# 2. LIVE SIMULATION (MOCK WEBSOCKET)
# ==========================================
def simulate_live_market(csv_file_path):
    print(f"--- STARTING LIVE SIMULATION ON {csv_file_path} ---")
    
    if not os.path.exists(csv_file_path):
        print(f"❌ ERROR: File '{csv_file_path}' not found.")
        print("Please upload 'master_fut_df.csv' to the current directory.")
        return None

    # Load Data
    try:
        print("Loading CSV (This may take a moment)...")
        df = pd.read_csv(csv_file_path)
        
        # --- ROBUST DATE PARSING ---
        print("Parsing timestamps...")
        
        # Case A: Separate Date and Time columns
        if 'Date' in df.columns and 'Time' in df.columns:
            # Try fast parsing first
            try:
                df['DateTime'] = pd.to_datetime(
                    df['Date'] + ' ' + df['Time'], 
                    format='%d/%m/%Y %H:%M:%S.%f', # Matches your data: 01/09/2025 08:46:42.278
                    errors='coerce'
                )
            except:
                # Fallback to flexible parsing
                 df['DateTime'] = pd.to_datetime(
                    df['Date'] + ' ' + df['Time'], 
                    dayfirst=True,
                    errors='coerce'
                )
                
        # Case B: Single DateTime column
        elif 'DateTime' in df.columns:
             df['DateTime'] = pd.to_datetime(df['DateTime'], dayfirst=True, errors='coerce')
        
        # Drop invalid rows
        initial_len = len(df)
        df = df.dropna(subset=['DateTime']).sort_values('DateTime').reset_index(drop=True)
        print(f"Loaded {len(df)} valid ticks (Dropped {initial_len - len(df)} bad rows).")
        print(f"Range: {df['DateTime'].min()} to {df['DateTime'].max()}")
        
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return None

    # Initialize the Brain
    brain = LiveKineticBrain(threshold=KINETIC_THRESHOLD)
    
    trades = []
    in_trade = False
    entry_data = {} 
    
    total_ticks = len(df)
    print("\nProcessing Tick Stream...")
    
    # --- THE EVENT LOOP ---
    for i, row in df.iterrows():
        if i % 100000 == 0:
            sys.stdout.write(f"\rProgress: {i/total_ticks*100:.1f}%")
            sys.stdout.flush()
            
        # Extract fields (Handle aliases)
        ltp = row.get('LTP') or row.get('LastTradedPrice') or row.get('Close')
        vol = row.get('Volume') or row.get('LTQ') or row.get('OpenInterest')
        ts = row['DateTime']
        
        if pd.isna(ltp) or pd.isna(vol): continue
            
        curr_time = ts.time()
        
        # 1. FEED THE BRAIN (Always feed to keep buffer current)
        signal_fired = brain.process_tick(ltp, vol)
        
        # 2. TRADE MANAGEMENT
        if in_trade:
            time_in_trade = (ts - entry_data['Entry_Time']).total_seconds()
            should_exit = False
            exit_reason = ""
            
            # Condition A: Time Limit
            if time_in_trade >= HOLD_SECONDS:
                should_exit = True
                exit_reason = "TIME_LIMIT"
            
            # Condition B: Market Close Force Exit (15:30)
            if curr_time >= MARKET_END:
                should_exit = True
                exit_reason = "MARKET_CLOSE"
            
            if should_exit:
                exit_price = ltp
                # FIX: Use Absolute Difference to measure Magnitude (Pure Volatility)
                # This assumes we correctly guessed direction (Perfect Trend Following or Straddle)
                pnl = abs(exit_price - entry_data['Entry_Price'])
                
                trades.append({
                    'Entry_Time': entry_data['Entry_Time'],
                    'Entry_Price': entry_data['Entry_Price'],
                    'Exit_Time': ts,
                    'Exit_Price': exit_price,
                    'Score': entry_data['Score'],
                    'PnL_Points': pnl,
                    'Reason': exit_reason
                })
                
                in_trade = False
                entry_data = {}

        # 3. ENTRY LOGIC
        else:
            # Only enter if within Market Hours (09:15 - 15:15)
            # Also ensure we are not entering right before close
            if curr_time >= MARKET_START and curr_time < datetime.time(15, 15):
                if signal_fired:
                    in_trade = True
                    entry_data = {
                        'Entry_Time': ts,
                        'Entry_Price': ltp,
                        'Score': brain.last_score
                    }
    
    # Force close if still open at end of file
    if in_trade:
         trades.append({
            'Entry_Time': entry_data['Entry_Time'],
            'Entry_Price': entry_data['Entry_Price'],
            'Exit_Time': df.iloc[-1]['DateTime'],
            'Exit_Price': df.iloc[-1]['LTP'], 
            'Score': entry_data['Score'],
            'PnL_Points': abs(df.iloc[-1]['LTP'] - entry_data['Entry_Price']),
            'Reason': "END_OF_DATA"
        })

    print(f"\n\n--- SIMULATION ENDED. Total Trades: {len(trades)} ---")
    
    # Results
    if trades:
        trade_df = pd.DataFrame(trades)
        
        total_pnl = trade_df['PnL_Points'].sum()
        # Win Rate is redundant here as magnitude is always >= 0, but we can check > 0
        win_rate = (trade_df['PnL_Points'] > 0).mean() * 100
        
        print("\n=== MAGNITUDE RESULTS SUMMARY ===")
        print(f"Total Magnitude Captured (Points): {total_pnl:.2f}")
        print(f"Total Magnitude Value (INR):       ₹{total_pnl * 75:,.2f}")
        print(f"Non-Zero Moves:                    {win_rate:.2f}%")
        print(f"Avg Move per Signal:               {trade_df['PnL_Points'].mean():.2f} pts")
        
        print("\n=== FIRST 5 SIGNALS ===")
        print(trade_df.head())
        return trade_df
    else:
        print("No trades generated.")
        return pd.DataFrame()

if __name__ == "__main__":
    FILENAME = "master_fut_df.csv"
    df_results = simulate_live_market(FILENAME)

--- STARTING LIVE SIMULATION ON master_fut_df.csv ---
Loading CSV (This may take a moment)...
Parsing timestamps...
Loaded 1213548 valid ticks (Dropped 0 bad rows).
Range: 2025-09-01 08:46:42.278000 to 2025-11-13 15:29:36.267000

Processing Tick Stream...
Progress: 98.9%

--- SIMULATION ENDED. Total Trades: 819 ---

=== MAGNITUDE RESULTS SUMMARY ===
Total Magnitude Captured (Points): 13613.70
Total Magnitude Value (INR):       ₹1,021,027.50
Non-Zero Moves:                    99.15%
Avg Move per Signal:               16.62 pts

=== FIRST 5 SIGNALS ===
               Entry_Time  Entry_Price               Exit_Time  Exit_Price  \
0 2025-09-01 09:16:04.406      24637.3 2025-09-01 09:31:04.506     24665.1   
1 2025-09-01 09:31:36.229      24673.4 2025-09-01 09:46:36.292     24677.3   
2 2025-09-01 09:46:37.146      24680.0 2025-09-01 10:01:38.278     24680.0   
3 2025-09-01 10:01:44.319      24680.0 2025-09-01 10:16:44.536     24659.5   
4 2025-09-01 10:16:54.795      24660.0 2025-09-01 10:

In [62]:
df_results.to_csv('trades_df')

In [63]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

# Contract Expiries
EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# --- SMART STRATEGY PARAMS ---
KINETIC_FLOOR = 75000      # Eliminate Noise (<75k)
KINETIC_CEILING = 500000   # Eliminate Exhaustion (>500k)

# Tiered Targets
HIGH_ENERGY_THRESH = 100000
TARGET_LOW_ENERGY = 20.0   # Target for 75k-100k scores
TARGET_HIGH_ENERGY = 40.0  # Target for 100k-500k scores

MAX_HOLD_SECONDS = 900     # 15 Mins max hold
COOLDOWN_SECONDS = 900     # 15 Mins cooldown after trade
COST_PER_TRADE = 1.0       # Points slippage/comm
LOT_SIZE = 75

# ==========================================
# 1. S3 & DATA UTILITIES
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year
    month = date_obj.month
    day = date_obj.day
    ts = get_trading_symbol(date_obj)
    
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(
                df['Date'].astype(str) + " " + df['Time'].astype(str), 
                dayfirst=True, errors='coerce'
            )
        
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
            
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except Exception as e:
        return None

# ==========================================
# 2. KINETIC BRAIN (FILTERED)
# ==========================================
class KineticBrain:
    def __init__(self):
        self.tick_buffer = deque(maxlen=50) 
        self.last_score = 0

    def process_tick(self, ltp, cumulative_volume):
        self.tick_buffer.append([ltp, cumulative_volume])
        
        if len(self.tick_buffer) < 50:
            return 0 
        
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        
        displacement = abs(prices[-1] - prices[0])
        total_vol = np.sum(trade_vol)
        
        score = total_vol / (displacement + 0.05)
        self.last_score = score
        
        # --- SMART FILTER ---
        if score >= KINETIC_FLOOR and score <= KINETIC_CEILING:
            return score # Valid Signal
            
        return 0 # Noise or Exhaustion

# ==========================================
# 3. SMART MAGNITUDE ENGINE
# ==========================================
def run_day(df):
    brain = KineticBrain()
    
    # State
    state = 'SEARCHING' # SEARCHING, MONITORING, COOLDOWN
    
    entry_time = None
    entry_price = 0.0
    active_target = 0.0
    active_score = 0.0
    
    cooldown_start = None
    trades = []
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        # --- STATE: COOLDOWN ---
        if state == 'COOLDOWN':
            if (ts - cooldown_start).total_seconds() > COOLDOWN_SECONDS:
                state = 'SEARCHING'
                brain.tick_buffer.clear() # Reset brain
            continue

        # --- STATE: SEARCHING ---
        if state == 'SEARCHING':
            score = brain.process_tick(ltp, vol)
            
            if score > 0:
                # SIGNAL FIRED
                state = 'MONITORING'
                entry_time = ts
                entry_price = ltp
                active_score = score
                
                # Set Smart Target
                if score >= HIGH_ENERGY_THRESH:
                    active_target = TARGET_HIGH_ENERGY
                else:
                    active_target = TARGET_LOW_ENERGY
                    
        # --- STATE: MONITORING (IN TRADE) ---
        elif state == 'MONITORING':
            elapsed = (ts - entry_time).total_seconds()
            current_move = abs(ltp - entry_price)
            
            # Check Target Hit (Magnitude Captured)
            if current_move >= active_target:
                # Success! We captured the full target move
                trades.append({
                    'Entry_Time': entry_time,
                    'Score': active_score,
                    'Target': active_target,
                    'Captured_Pts': active_target - COST_PER_TRADE, # Net PnL
                    'Outcome': 'TARGET_HIT'
                })
                state = 'COOLDOWN'
                cooldown_start = ts
                continue
                
            # Check Time Limit
            if elapsed >= MAX_HOLD_SECONDS:
                # Time's up! Capture whatever move exists now
                final_move = current_move
                trades.append({
                    'Entry_Time': entry_time,
                    'Score': active_score,
                    'Target': active_target,
                    'Captured_Pts': final_move - COST_PER_TRADE,
                    'Outcome': 'TIME_EXIT'
                })
                state = 'COOLDOWN'
                cooldown_start = ts
                
    return trades

# ==========================================
# 4. MAIN LOOP & REPORTING
# ==========================================
def main():
    print(f"Running SMART KINETIC HUNTER")
    print(f"Range: {START_DATE} to {END_DATE}")
    print(f"Filter: {KINETIC_FLOOR/1000:.1f}k < Score < {KINETIC_CEILING/1000:.0f}k")
    print(f"Targets: <100k->{TARGET_LOW_ENERGY}pts | >100k->{TARGET_HIGH_ENERGY}pts")
    print("-" * 75)
    print(f"{'Date':<12} | {'Trades':<6} | {'Pts Captured':<12} | {'Avg/Trade':<10} | {'Status'}")
    print("-" * 75)
    
    all_trades = []
    current_date = START_DATE
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            
            if daily_trades:
                # Convert to DF for easier summation if needed, or just sum dicts
                pts = sum(t['Captured_Pts'] for t in daily_trades)
                count = len(daily_trades)
                avg = pts / count
                
                status = "PROFIT" if pts > 0 else "LOSS"
                print(f"{current_date} | {count:<6} | {pts:>9.2f} pts  | {avg:>9.2f}  | {status}")
                
                all_trades.extend(daily_trades)
            else:
                 pass
                 # print(f"{current_date} | 0      |      0.00 pts  |      0.00  | NO SIGNALS")
        
        current_date += datetime.timedelta(days=1)
        
    # --- FINAL REPORT ---
    if not all_trades:
        print("No trades found.")
        return

    df_res = pd.DataFrame(all_trades)
    
    total_pts = df_res['Captured_Pts'].sum()
    avg_trade = df_res['Captured_Pts'].mean()
    hit_rate = (df_res['Outcome'] == 'TARGET_HIT').mean() * 100
    
    print("\n" + "="*50)
    print(f"SMART MAGNITUDE SUMMARY (N={len(df_res)})")
    print("="*50)
    print(f"Total Points Captured: {total_pts:.2f}")
    print(f"Total Value (INR):     ₹{total_pts * LOT_SIZE:,.2f}")
    print(f"Average Pts/Trade:     {avg_trade:.2f}")
    print(f"Target Hit Rate:       {hit_rate:.1f}%")
    
    print("-" * 50)
    # Breakdown by Score Tier
    df_res['Tier'] = np.where(df_res['Score'] >= HIGH_ENERGY_THRESH, 'High Energy (>100k)', 'Med Energy (<100k)')
    breakdown = df_res.groupby('Tier').agg({
        'Captured_Pts': ['count', 'mean', 'sum'],
        'Outcome': lambda x: (x == 'TARGET_HIT').mean() * 100
    })
    breakdown.columns = ['Trades', 'Avg Pts', 'Total Pts', 'Target Hit %']
    print(breakdown.round(2))
    
    # Save for analysis
    df_res.to_csv("smart_kinetic_results.csv", index=False)
    print(f"\nDetailed logs saved to: smart_kinetic_results.csv")

if __name__ == "__main__":
    main()

Running SMART KINETIC HUNTER
Range: 2025-09-01 to 2025-11-26
Filter: 75.0k < Score < 500k
Targets: <100k->20.0pts | >100k->40.0pts
---------------------------------------------------------------------------
Date         | Trades | Pts Captured | Avg/Trade  | Status
---------------------------------------------------------------------------
2025-09-01 | 12     |     73.60 pts  |      6.13  | PROFIT
2025-09-02 | 12     |    192.80 pts  |     16.07  | PROFIT
2025-09-03 | 9      |    105.80 pts  |     11.76  | PROFIT
2025-09-04 | 12     |    212.90 pts  |     17.74  | PROFIT
2025-09-05 | 13     |    133.40 pts  |     10.26  | PROFIT
2025-09-08 | 11     |    151.00 pts  |     13.73  | PROFIT
2025-09-09 | 9      |     76.70 pts  |      8.52  | PROFIT
2025-09-10 | 12     |    130.80 pts  |     10.90  | PROFIT
2025-09-11 | 9      |     53.70 pts  |      5.97  | PROFIT
2025-09-12 | 12     |     71.40 pts  |      5.95  | PROFIT
2025-09-18 | 11     |    125.80 pts  |     11.44  | PROFIT
2025-09-2

In [4]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO
from numba import njit

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# --- STRATEGY: VISIBILITY GRAPH KINETIC ---
KINETIC_THRESHOLD = 37500     
HVG_WINDOW = 50               # Lookback for Graph Construction

# GRAPH FILTER
# If Mean Degree is HIGH, price is tangled (Chop) -> AVOID
# If Mean Degree is LOW, price is free (Trend) -> ATTACK
DEGREE_THRESHOLD = 3.5        # Empirical cutoff for HVG Density

# Trade Management (Standard Trend Following)
STOP_LOSS_POINTS = 20.0       
TAKE_PROFIT_POINTS = 60.0     # 1:3 Risk Reward
MAX_HOLD_SECONDS = 1800       
COOLDOWN_SECONDS = 300        # 5 Mins (Let the graph reset)
COST_PER_TRADE = 1.0          

# Capital
CAPITAL = 1000000             
MARGIN_PER_LOT = 150000       
LOT_SIZE = 75
NUM_LOTS = int(CAPITAL / MARGIN_PER_LOT) 
QTY = NUM_LOTS * LOT_SIZE     

# ==========================================
# 1. NUMBA GRAPH ENGINE (HVG)
# ==========================================
@njit
def calculate_hvg_mean_degree(prices):
    """
    Calculates the Mean Degree of a Horizontal Visibility Graph.
    O(N^2) worst case, but fast on small windows (50).
    """
    n = len(prices)
    if n < 2: return 0.0
    
    total_degree = 0
    
    # Check visibility for every node i
    for i in range(n):
        current_degree = 0
        
        # Look backwards
        if i > 0:
            max_h = -99999999.0
            for j in range(i - 1, -1, -1):
                # Visibility condition: prices[k] < min(prices[i], prices[j])
                # For HVG: Two nodes i and j see each other if all intermediate nodes are smaller
                if j == i - 1: # Immediate neighbor always visible
                    current_degree += 1
                    max_h = prices[j]
                elif prices[j] > max_h:
                    current_degree += 1
                    max_h = prices[j]
                
                # Optimization: If we hit a wall bigger than current node, we can't see past it?
                # For HVG specifically: node i sees node j if prices[k] < prices[i] AND prices[k] < prices[j]
                # Actually simple HVG: i and j see each other if y_k < min(y_i, y_j) for all k in between.
                # My optimized logic above approximates "looking back" line of sight.
                if prices[j] >= prices[i]: break
                
        # Look forwards
        if i < n - 1:
            max_h = -99999999.0
            for j in range(i + 1, n):
                if j == i + 1:
                    current_degree += 1
                    max_h = prices[j]
                elif prices[j] > max_h:
                    current_degree += 1
                    max_h = prices[j]
                
                if prices[j] >= prices[i]: break
        
        total_degree += current_degree

    return total_degree / n

# ==========================================
# 2. KINETIC GRAPH BRAIN
# ==========================================
class KineticGraphBrain:
    def __init__(self):
        self.tick_buffer = deque(maxlen=HVG_WINDOW) 
        self.last_score = 0
        self.last_degree = 0.0

    def process_tick(self, ltp, cumulative_volume):
        self.tick_buffer.append([ltp, cumulative_volume])
        
        if len(self.tick_buffer) < HVG_WINDOW: return 0 
        
        data = np.array(self.tick_buffer)
        prices = data[:, 0]
        vols = data[:, 1]
        
        # 1. Calculate Kinetic (Energy)
        vol_diff = np.diff(vols)
        trade_vol = np.where(vol_diff > 0, vol_diff, 0)
        displacement = abs(prices[-1] - prices[0])
        kinetic_score = np.sum(trade_vol) / (displacement + 0.05)
        self.last_score = kinetic_score
        
        # 2. Check Energy Threshold
        if kinetic_score > KINETIC_THRESHOLD:
            
            # 3. Calculate Graph Topology (Structure)
            # We pass contiguous numpy array of prices to Numba
            mean_degree = calculate_hvg_mean_degree(prices)
            self.last_degree = mean_degree
            
            # 4. The "Structural Break" Signal
            # Low Degree = Low Connectivity = Unstable/Trend
            # High Degree = High Connectivity = Mean Reversion/Chop
            
            if mean_degree < DEGREE_THRESHOLD:
                # Structure is broken -> Follow Momentum
                direction = 1 if prices[-1] > prices[0] else -1
                return direction
                
        return 0

# ==========================================
# 3. EXECUTION ENGINE
# ==========================================
def run_day(df):
    brain = KineticGraphBrain()
    
    state = 'SEARCHING'
    
    entry_price = 0.0
    entry_time = None
    direction = 0 
    
    cooldown_start = None
    trades = []
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        if state == 'COOLDOWN':
            if (ts - cooldown_start).total_seconds() > COOLDOWN_SECONDS:
                state = 'SEARCHING'
                brain.tick_buffer.clear()
            continue

        if state == 'SEARCHING':
            # Returns 1 (Long), -1 (Short), or 0
            sig = brain.process_tick(ltp, vol)
            
            if sig != 0:
                state = 'IN_TRADE'
                direction = sig
                entry_price = ltp
                entry_time = ts
                
        elif state == 'IN_TRADE':
            elapsed = (ts - entry_time).total_seconds()
            pnl = 0
            exit_reason = None
            
            # Calc PnL
            if direction == 1: pnl = ltp - entry_price
            else: pnl = entry_price - ltp
            
            # Exits
            if pnl <= -STOP_LOSS_POINTS: exit_reason = "SL"
            elif pnl >= TAKE_PROFIT_POINTS: exit_reason = "TP"
            elif elapsed >= MAX_HOLD_SECONDS: exit_reason = "TIME"
            
            if exit_reason:
                net_pts = pnl - COST_PER_TRADE
                trades.append({
                    'Entry_Time': entry_time,
                    'Type': 'LONG' if direction == 1 else 'SHORT',
                    'Net_Pts': net_pts,
                    'PnL_INR': net_pts * QTY,
                    'Reason': exit_reason,
                    'Degree': brain.last_degree
                })
                state = 'COOLDOWN'
                cooldown_start = ts
                
    return trades

# ==========================================
# 4. MAIN LOOP
# ==========================================
# S3 Get Data Function (Reused)
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year; month = date_obj.month; day = date_obj.day
    ts = get_trading_symbol(date_obj)
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + " " + df['Time'].astype(str), dayfirst=True, errors='coerce')
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except: return None

def main():
    print(f"Running KINETIC GRAPH TOPOLOGY STRATEGY")
    print(f"Logic: Kinetic > {KINETIC_THRESHOLD} AND HVG Degree < {DEGREE_THRESHOLD}")
    print(f"Hypothesis: Low Graph Degree = Structural Breakout (Trend)")
    print("-" * 80)
    
    all_trades = []
    current_date = START_DATE
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            
            if daily_trades:
                daily_df = pd.DataFrame(daily_trades)
                pnl = daily_df['PnL_INR'].sum()
                wins = len(daily_df[daily_df['Net_Pts'] > 0])
                win_rate = (wins / len(daily_df)) * 100
                print(f"{current_date} | {len(daily_trades):<6} | {win_rate:<5.1f}% | ₹{pnl:>12,.2f}")
                all_trades.extend(daily_trades)
            else:
                print(f"{current_date} | 0      | -     | ₹0.00")
        
        current_date += datetime.timedelta(days=1)
        
    if not all_trades:
        print("No trades found.")
        return

    df_res = pd.DataFrame(all_trades)
    
    total_pnl = df_res['PnL_INR'].sum()
    roi = (total_pnl / CAPITAL) * 100
    win_rate = (len(df_res[df_res['Net_Pts'] > 0]) / len(df_res)) * 100
    
    print("\n" + "="*50)
    print(f"FINAL PERFORMANCE SUMMARY")
    print("="*50)
    print(f"Total Net Profit:      ₹{total_pnl:,.2f}")
    print(f"Return on Capital:     {roi:.2f}%")
    print(f"Total Trades:          {len(df_res)}")
    print(f"Win Rate:              {win_rate:.2f}%")
    print(f"Avg Profit per Trade:  ₹{df_res['PnL_INR'].mean():,.2f}")
    
    df_res.to_csv("kinetic_graph_results.csv", index=False)

if __name__ == "__main__":
    main()

Running KINETIC GRAPH TOPOLOGY STRATEGY
Logic: Kinetic > 37500 AND HVG Degree < 3.5
Hypothesis: Low Graph Degree = Structural Breakout (Trend)
--------------------------------------------------------------------------------
2025-09-01 | 13     | 15.4 % | ₹  -60,390.00
2025-09-02 | 15     | 40.0 % | ₹   15,345.00
2025-09-03 | 11     | 27.3 % | ₹  -17,775.00
2025-09-04 | 11     | 72.7 % | ₹   67,995.00
2025-09-05 | 18     | 16.7 % | ₹ -108,495.00
2025-09-08 | 14     | 35.7 % | ₹   -9,000.00
2025-09-09 | 11     | 18.2 % | ₹  -40,365.00
2025-09-10 | 12     | 66.7 % | ₹   31,950.00
2025-09-11 | 11     | 54.5 % | ₹  -23,985.00
2025-09-12 | 12     | 41.7 % | ₹  -19,755.00
2025-09-18 | 11     | 63.6 % | ₹   14,310.00
2025-09-22 | 12     | 25.0 % | ₹  -50,175.00
2025-09-23 | 13     | 30.8 % | ₹  -27,000.00
2025-09-25 | 13     | 61.5 % | ₹   80,415.00
2025-10-03 | 6      | 16.7 % | ₹  -22,320.00
2025-10-06 | 11     | 27.3 % | ₹  -45,630.00
2025-10-07 | 13     | 53.8 % | ₹   47,160.00
2025-10-08 

In [9]:
import boto3
import pandas as pd
import numpy as np
import os
import datetime
from collections import deque
from io import BytesIO
from numba import njit

# ==========================================
# CONFIGURATION
# ==========================================
BUCKET = "live-market-data"
SYMBOL = "NIFTY"

START_DATE = datetime.date(2025, 9, 1)
END_DATE = datetime.date(2025, 11, 26)

EXPIRY_SEP = datetime.date(2025, 9, 25)
EXPIRY_OCT = datetime.date(2025, 10, 30)
EXPIRY_NOV = datetime.date(2025, 11, 27)

# --- STRATEGY: INVERSE LIQUIDITY GRAB ---
KINETIC_THRESHOLD = 100000   
HVG_WINDOW = 50               
DEGREE_THRESHOLD = 3.5        

# PHASE SPACE PARAMETERS
EMBEDDING_DIM = 3             
EMBEDDING_DELAY = 5           
LYAPUNOV_THRESHOLD = 0.15     
MOMENTUM_ASYMMETRY_MIN = 0.6  

# NEW: LIQUIDITY DETECTION
VOLUME_PRICE_DIVERGENCE_MIN = 0.4  # High vol, low price move = trap
INVERSE_MODE = True                 # TRADE OPPOSITE OF SIGNAL

# Trade Management
STOP_LOSS_POINTS = 20.0       
TAKE_PROFIT_POINTS = 60.0     
MAX_HOLD_SECONDS = 1800       
COOLDOWN_SECONDS = 300        
COST_PER_TRADE = 1.0          

# Capital
CAPITAL = 1000000             
MARGIN_PER_LOT = 150000       
LOT_SIZE = 75
NUM_LOTS = int(CAPITAL / MARGIN_PER_LOT) 
QTY = NUM_LOTS * LOT_SIZE     

# ==========================================
# 1. NUMBA GRAPH ENGINE (HVG)
# ==========================================
@njit
def calculate_hvg_mean_degree(prices):
    n = len(prices)
    if n < 2: return 0.0
    
    total_degree = 0
    
    for i in range(n):
        current_degree = 0
        
        if i > 0:
            max_h = -99999999.0
            for j in range(i - 1, -1, -1):
                if j == i - 1:
                    current_degree += 1
                    max_h = prices[j]
                elif prices[j] > max_h:
                    current_degree += 1
                    max_h = prices[j]
                
                if prices[j] >= prices[i]: break
                
        if i < n - 1:
            max_h = -99999999.0
            for j in range(i + 1, n):
                if j == i + 1:
                    current_degree += 1
                    max_h = prices[j]
                elif prices[j] > max_h:
                    current_degree += 1
                    max_h = prices[j]
                
                if prices[j] >= prices[i]: break
        
        total_degree += current_degree

    return total_degree / n

# ==========================================
# 2. PHASE SPACE TOOLKIT
# ==========================================
@njit
def calculate_lyapunov_local(prices, delay=5, steps=10):
    n = len(prices)
    if n < delay + steps + 5:
        return 0.0
    
    ref_idx = n - delay - steps - 1
    ref_point = prices[ref_idx]
    
    min_dist = 99999999.0
    nearest_idx = -1
    
    for i in range(10, n - delay - steps - 10):
        if abs(i - ref_idx) < 5:
            continue
        dist = abs(prices[i] - ref_point)
        if dist < min_dist and dist > 0.01:
            min_dist = dist
            nearest_idx = i
    
    if nearest_idx == -1:
        return 0.0
    
    initial_dist = abs(prices[ref_idx] - prices[nearest_idx])
    final_dist = abs(prices[ref_idx + steps] - prices[nearest_idx + steps])
    
    if initial_dist < 0.01 or final_dist < 0.01:
        return 0.0
    
    lyap = np.log(final_dist / initial_dist) / steps
    return lyap

@njit
def volume_momentum_asymmetry(prices, volumes, window=20):
    n = len(prices)
    if n < window + 1:
        return 0.0, 0.0
    
    up_momentum = 0.0
    down_momentum = 0.0
    
    start_idx = n - window
    
    for i in range(start_idx, n - 1):
        price_change = prices[i + 1] - prices[i]
        vol_weight = volumes[i + 1]
        
        if price_change > 0:
            up_momentum += price_change * vol_weight
        elif price_change < 0:
            down_momentum += abs(price_change) * vol_weight
    
    total_momentum = up_momentum + down_momentum
    if total_momentum < 0.01:
        return 0.0, 0.0
    
    direction = (up_momentum - down_momentum) / total_momentum
    confidence = max(up_momentum, down_momentum) / total_momentum
    
    return direction, confidence

@njit
def calculate_order_flow_pressure(volumes, prices, window=15):
    n = len(volumes)
    if n < window + 1:
        return 0.0
    
    buy_volume = 0.0
    sell_volume = 0.0
    
    start_idx = n - window
    
    for i in range(start_idx + 1, n):
        vol_delta = volumes[i] - volumes[i - 1]
        if vol_delta <= 0:
            continue
            
        price_change = prices[i] - prices[i - 1]
        
        if price_change > 0:
            buy_volume += vol_delta
        elif price_change < 0:
            sell_volume += vol_delta
        else:
            buy_volume += vol_delta * 0.5
            sell_volume += vol_delta * 0.5
    
    total_volume = buy_volume + sell_volume
    if total_volume < 1:
        return 0.0
    
    return (buy_volume - sell_volume) / total_volume

@njit
def calculate_kinetic_energy(volumes, prices):
    n = len(volumes)
    if n < 2:
        return 0.0
    
    trade_vol_sum = 0.0
    for i in range(1, n):
        vol_diff = volumes[i] - volumes[i - 1]
        if vol_diff > 0:
            trade_vol_sum += vol_diff
    
    displacement = abs(prices[-1] - prices[0])
    kinetic = trade_vol_sum / (displacement + 0.05)
    
    return kinetic

@njit
def detect_volume_price_divergence(volumes, prices, window=20):
    """
    NEW: Detects when volume is high but price movement is low.
    This indicates accumulation/distribution (liquidity grab).
    Returns divergence_score: 0 (aligned) to 1 (maximum divergence)
    """
    n = len(volumes)
    if n < window + 1:
        return 0.0
    
    start_idx = n - window
    
    # Calculate volume intensity
    total_volume = 0.0
    for i in range(start_idx, n):
        vol_delta = volumes[i] - volumes[i - 1] if i > 0 else volumes[i]
        if vol_delta > 0:
            total_volume += vol_delta
    
    # Calculate price movement
    price_range = 0.0
    for i in range(start_idx + 1, n):
        price_range += abs(prices[i] - prices[i - 1])
    
    if price_range < 0.01 or total_volume < 1:
        return 0.0
    
    # Normalized metrics
    avg_vol = total_volume / window
    avg_price_move = price_range / window
    
    # High volume, low price move = HIGH divergence (liquidity trap)
    # We want: high avg_vol relative to avg_price_move
    vol_price_ratio = avg_vol / (avg_price_move + 0.01)
    
    # Normalize to 0-1 range (empirically tuned)
    divergence = min(1.0, vol_price_ratio / 50000.0)
    
    return divergence

# ==========================================
# 3. INVERSE LIQUIDITY BRAIN
# ==========================================
class InverseLiquidityBrain:
    def __init__(self):
        self.tick_buffer = deque(maxlen=HVG_WINDOW) 
        self.last_score = 0
        self.last_degree = 0.0
        self.last_lyapunov = 0.0
        self.last_direction_score = 0.0
        self.last_confidence = 0.0
        self.last_order_flow = 0.0
        self.last_divergence = 0.0

    def process_tick(self, ltp, cumulative_volume):
        self.tick_buffer.append([ltp, cumulative_volume])
        
        if len(self.tick_buffer) < HVG_WINDOW: 
            return 0, {}
        
        data = np.array(self.tick_buffer, dtype=np.float64)
        prices = np.ascontiguousarray(data[:, 0])
        vols = np.ascontiguousarray(data[:, 1])
        
        # 1. Calculate Kinetic (Energy)
        kinetic_score = calculate_kinetic_energy(vols, prices)
        self.last_score = kinetic_score
        
        if kinetic_score <= KINETIC_THRESHOLD:
            return 0, {}
            
        # 2. Calculate Graph Topology
        mean_degree = calculate_hvg_mean_degree(prices)
        self.last_degree = mean_degree
        
        if mean_degree >= DEGREE_THRESHOLD:
            return 0, {}
        
        # 3. Lyapunov Exponent
        lyap = calculate_lyapunov_local(prices, delay=EMBEDDING_DELAY, steps=10)
        self.last_lyapunov = lyap
        
        # 4. Volume-Weighted Momentum
        dir_score, confidence = volume_momentum_asymmetry(prices, vols, window=30)
        self.last_direction_score = dir_score
        self.last_confidence = confidence
        
        # 5. Order Flow Pressure
        order_flow = calculate_order_flow_pressure(vols, prices, window=20)
        self.last_order_flow = order_flow
        
        # 6. NEW: Volume-Price Divergence Detection
        divergence = detect_volume_price_divergence(vols, prices, window=30)
        self.last_divergence = divergence
        
        # ============================================
        # LIQUIDITY GRAB LOGIC
        # ============================================
        
        # Require high confidence
        if confidence < MOMENTUM_ASYMMETRY_MIN:
            return 0, {}
        
        # Lyapunov filter
        if lyap < 0.05 or lyap > 0.3:
            return 0, {}
        
        # NEW: Check for volume-price divergence
        # High divergence = liquidity trap = INVERSE THE SIGNAL
        is_liquidity_trap = divergence > VOLUME_PRICE_DIVERGENCE_MIN
        
        # Calculate apparent direction
        combined_signal = (0.6 * dir_score) + (0.4 * order_flow)
        
        if abs(combined_signal) < 0.5:
            return 0, {}
        
        apparent_direction = 1 if combined_signal > 0 else -1
        
        # THE EDGE: If liquidity trap detected, fade the move
        if INVERSE_MODE and is_liquidity_trap:
            actual_direction = -1 * apparent_direction
            trade_logic = "INVERSE"
        else:
            actual_direction = apparent_direction
            trade_logic = "DIRECT"
        
        metrics = {
            'kinetic': kinetic_score,
            'degree': mean_degree,
            'lyapunov': lyap,
            'momentum_dir': dir_score,
            'confidence': confidence,
            'order_flow': order_flow,
            'combined_signal': combined_signal,
            'divergence': divergence,
            'is_trap': is_liquidity_trap,
            'trade_logic': trade_logic
        }
        
        return actual_direction, metrics

# ==========================================
# 4. EXECUTION ENGINE
# ==========================================
def run_day(df):
    brain = InverseLiquidityBrain()
    
    state = 'SEARCHING'
    
    entry_price = 0.0
    entry_time = None
    direction = 0 
    entry_metrics = {}
    
    cooldown_start = None
    trades = []
    
    for row in df.itertuples():
        ts = row.DateTime
        ltp = row.LTP
        vol = row.Volume
        
        if state == 'COOLDOWN':
            if (ts - cooldown_start).total_seconds() > COOLDOWN_SECONDS:
                state = 'SEARCHING'
                brain.tick_buffer.clear()
            continue

        if state == 'SEARCHING':
            sig, metrics = brain.process_tick(ltp, vol)
            
            if sig != 0:
                state = 'IN_TRADE'
                direction = sig
                entry_price = ltp
                entry_time = ts
                entry_metrics = metrics
                
        elif state == 'IN_TRADE':
            elapsed = (ts - entry_time).total_seconds()
            pnl = 0
            exit_reason = None
            
            if direction == 1: 
                pnl = ltp - entry_price
            else: 
                pnl = entry_price - ltp
            
            if pnl <= -STOP_LOSS_POINTS: 
                exit_reason = "SL"
            elif pnl >= TAKE_PROFIT_POINTS: 
                exit_reason = "TP"
            elif elapsed >= MAX_HOLD_SECONDS: 
                exit_reason = "TIME"
            
            if exit_reason:
                net_pts = pnl - COST_PER_TRADE
                
                trade_record = {
                    'Entry_Time': entry_time,
                    'Type': 'LONG' if direction == 1 else 'SHORT',
                    'Net_Pts': net_pts,
                    'PnL_INR': net_pts * QTY,
                    'Reason': exit_reason,
                    'Degree': entry_metrics.get('degree', 0),
                    'Lyapunov': entry_metrics.get('lyapunov', 0),
                    'Momentum': entry_metrics.get('momentum_dir', 0),
                    'Confidence': entry_metrics.get('confidence', 0),
                    'OrderFlow': entry_metrics.get('order_flow', 0),
                    'Signal': entry_metrics.get('combined_signal', 0),
                    'Divergence': entry_metrics.get('divergence', 0),
                    'IsTrap': entry_metrics.get('is_trap', False),
                    'Logic': entry_metrics.get('trade_logic', 'UNKNOWN')
                }
                trades.append(trade_record)
                
                state = 'COOLDOWN'
                cooldown_start = ts
                
    return trades

# ==========================================
# 5. MAIN LOOP
# ==========================================
def get_trading_symbol(current_date):
    if current_date <= EXPIRY_SEP: return "NIFTY25SEPFUT"
    elif current_date <= EXPIRY_OCT: return "NIFTY25OCTFUT"
    else: return "NIFTY25NOVFUT"

def get_data_for_date(date_obj):
    s3 = boto3.client("s3")
    year = date_obj.year; month = date_obj.month; day = date_obj.day
    ts = get_trading_symbol(date_obj)
    key = f"year={year}/month={month:02d}/day={day:02d}/Futures/{SYMBOL}/{ts}.parquet"
    try:
        obj = s3.get_object(Bucket=BUCKET, Key=key)
        df = pd.read_parquet(BytesIO(obj["Body"].read()))
        if 'DateTime' not in df.columns:
            df['DateTime'] = pd.to_datetime(df['Date'].astype(str) + " " + df['Time'].astype(str), dayfirst=True, errors='coerce')
        col_map = {'LastTradedPrice': 'LTP', 'Close': 'LTP'}
        df.rename(columns=col_map, inplace=True)
        if 'Volume' not in df.columns:
            if 'OpenInterest' in df.columns: df['Volume'] = df['OpenInterest']
            elif 'LTQ' in df.columns: df['Volume'] = df['LTQ'] 
            else: df['Volume'] = 0
        df = df.dropna(subset=['DateTime', 'LTP']).sort_values('DateTime').reset_index(drop=True)
        return df
    except: 
        return None

def main():
    mode_str = "🔄 INVERSE LIQUIDITY GRAB" if INVERSE_MODE else "➡️ DIRECT MOMENTUM"
    print(f"🚀 {mode_str} STRATEGY")
    print(f"Logic: Fade high-volume, low-displacement moves")
    print(f"  - Kinetic Threshold: {KINETIC_THRESHOLD}")
    print(f"  - Graph Degree < {DEGREE_THRESHOLD}")
    print(f"  - Divergence > {VOLUME_PRICE_DIVERGENCE_MIN} = TRAP")
    print(f"  - Inverse Mode: {INVERSE_MODE}")
    print("-" * 80)
    
    all_trades = []
    current_date = START_DATE
    
    while current_date <= END_DATE:
        if current_date.weekday() >= 5:
            current_date += datetime.timedelta(days=1)
            continue
            
        df = get_data_for_date(current_date)
        
        if df is not None and not df.empty:
            daily_trades = run_day(df)
            
            if daily_trades:
                daily_df = pd.DataFrame(daily_trades)
                pnl = daily_df['PnL_INR'].sum()
                wins = len(daily_df[daily_df['Net_Pts'] > 0])
                win_rate = (wins / len(daily_df)) * 100
                print(f"{current_date} | {len(daily_trades):<6} | {win_rate:<5.1f}% | ₹{pnl:>12,.2f}")
                all_trades.extend(daily_trades)
            else:
                print(f"{current_date} | 0      | -     | ₹0.00")
        
        current_date += datetime.timedelta(days=1)
        
    if not all_trades:
        print("\n❌ No trades found.")
        return

    df_res = pd.DataFrame(all_trades)
    
    total_pnl = df_res['PnL_INR'].sum()
    roi = (total_pnl / CAPITAL) * 100
    win_rate = (len(df_res[df_res['Net_Pts'] > 0]) / len(df_res)) * 100
    
    print("\n" + "="*50)
    print(f"💰 FINAL PERFORMANCE SUMMARY")
    print("="*50)
    print(f"Total Net Profit:      ₹{total_pnl:,.2f}")
    print(f"Return on Capital:     {roi:.2f}%")
    print(f"Total Trades:          {len(df_res)}")
    print(f"Win Rate:              {win_rate:.2f}%")
    print(f"Avg Profit per Trade:  ₹{df_res['PnL_INR'].mean():,.2f}")
    
    # Divergence analytics
    print(f"\n📊 LIQUIDITY TRAP METRICS:")
    print(f"Avg Divergence Score:  {df_res['Divergence'].mean():.3f}")
    trap_trades = df_res[df_res['IsTrap'] == True]
    direct_trades = df_res[df_res['IsTrap'] == False]
    
    if len(trap_trades) > 0:
        trap_wr = (len(trap_trades[trap_trades['Net_Pts'] > 0]) / len(trap_trades)) * 100
        print(f"TRAP TRADES: {len(trap_trades)} | {trap_wr:.1f}% WR | ₹{trap_trades['PnL_INR'].sum():,.2f}")
    
    if len(direct_trades) > 0:
        direct_wr = (len(direct_trades[direct_trades['Net_Pts'] > 0]) / len(direct_trades)) * 100
        print(f"DIRECT TRADES: {len(direct_trades)} | {direct_wr:.1f}% WR | ₹{direct_trades['PnL_INR'].sum():,.2f}")
    
    # Directional breakdown
    longs = df_res[df_res['Type'] == 'LONG']
    shorts = df_res[df_res['Type'] == 'SHORT']
    
    print(f"\n🎯 DIRECTIONAL BREAKDOWN:")
    if len(longs) > 0:
        long_wr = (len(longs[longs['Net_Pts'] > 0]) / len(longs)) * 100
        print(f"LONGS:  {len(longs)} trades | {long_wr:.1f}% WR | ₹{longs['PnL_INR'].sum():,.2f}")
    if len(shorts) > 0:
        short_wr = (len(shorts[shorts['Net_Pts'] > 0]) / len(shorts)) * 100
        print(f"SHORTS: {len(shorts)} trades | {short_wr:.1f}% WR | ₹{shorts['PnL_INR'].sum():,.2f}")
    
    df_res.to_csv("inverse_liquidity_results.csv", index=False)
    print(f"\n✅ Results saved to: inverse_liquidity_results.csv")

if __name__ == "__main__":
    main()

🚀 🔄 INVERSE LIQUIDITY GRAB STRATEGY
Logic: Fade high-volume, low-displacement moves
  - Kinetic Threshold: 100000
  - Graph Degree < 3.5
  - Divergence > 0.4 = TRAP
  - Inverse Mode: True
--------------------------------------------------------------------------------
2025-09-01 | 0      | -     | ₹0.00
2025-09-02 | 3      | 66.7 % | ₹   28,800.00
2025-09-03 | 0      | -     | ₹0.00
2025-09-04 | 2      | 0.0  % | ₹  -18,945.00
2025-09-05 | 1      | 100.0% | ₹    4,500.00
2025-09-08 | 0      | -     | ₹0.00
2025-09-09 | 0      | -     | ₹0.00
2025-09-10 | 0      | -     | ₹0.00
2025-09-11 | 0      | -     | ₹0.00
2025-09-12 | 1      | 100.0% | ₹      495.00
2025-09-18 | 0      | -     | ₹0.00
2025-09-22 | 1      | 100.0% | ₹    7,695.00
2025-09-23 | 3      | 33.3 % | ₹    6,435.00
2025-09-25 | 2      | 100.0% | ₹   30,150.00
2025-10-03 | 1      | 0.0  % | ₹   -9,540.00
2025-10-06 | 5      | 100.0% | ₹   36,045.00
2025-10-07 | 2      | 100.0% | ₹   46,485.00
2025-10-08 | 1      | 0.0  % 

In [12]:
df = pd.read_csv("master_fut_df.csv")
df

,Date,Time,Trading_Symbol,Instrument_Token,LTP,LTQ,Volume,Open_Interest,BestBid,BestAsk,BidSize,AskSize,Ticker
0,01/09/2025,08:46:42.278,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
1,01/09/2025,09:10:01.994,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
2,01/09/2025,09:15:01.722,NIFTY25SEPFUT,13568258,24590.0,300,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
3,01/09/2025,09:15:02.471,NIFTY25SEPFUT,13568258,24602.8,75,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
4,01/09/2025,09:15:02.975,NIFTY25SEPFUT,13568258,24595.7,75,5700,16610100,24595.7,24605.5,300,300,NIFTY25SEPFUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213543,13/11/2025,15:29:33.077,NIFTY25NOVFUT,9485826,25957.3,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213544,13/11/2025,15:29:34.659,NIFTY25NOVFUT,9485826,25957.5,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213545,13/11/2025,15:29:35.142,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT
1213546,13/11/2025,15:29:35.834,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT


In [13]:
df = pd.read_csv('master_fut_df.csv')
df

,Date,Time,Trading_Symbol,Instrument_Token,LTP,LTQ,Volume,Open_Interest,BestBid,BestAsk,BidSize,AskSize,Ticker
0,01/09/2025,08:46:42.278,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
1,01/09/2025,09:10:01.994,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
2,01/09/2025,09:15:01.722,NIFTY25SEPFUT,13568258,24590.0,300,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
3,01/09/2025,09:15:02.471,NIFTY25SEPFUT,13568258,24602.8,75,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
4,01/09/2025,09:15:02.975,NIFTY25SEPFUT,13568258,24595.7,75,5700,16610100,24595.7,24605.5,300,300,NIFTY25SEPFUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213543,13/11/2025,15:29:33.077,NIFTY25NOVFUT,9485826,25957.3,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213544,13/11/2025,15:29:34.659,NIFTY25NOVFUT,9485826,25957.5,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213545,13/11/2025,15:29:35.142,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT
1213546,13/11/2025,15:29:35.834,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT


In [14]:
df = pd.read_csv("master_fut_df.csv")
df

,Date,Time,Trading_Symbol,Instrument_Token,LTP,LTQ,Volume,Open_Interest,BestBid,BestAsk,BidSize,AskSize,Ticker
0,01/09/2025,08:46:42.278,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
1,01/09/2025,09:10:01.994,NIFTY25SEPFUT,13568258,24568.5,75,0,16610100,0.0,0.0,0,0,NIFTY25SEPFUT
2,01/09/2025,09:15:01.722,NIFTY25SEPFUT,13568258,24590.0,300,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
3,01/09/2025,09:15:02.471,NIFTY25SEPFUT,13568258,24602.8,75,1050,16610100,24586.6,24597.3,300,300,NIFTY25SEPFUT
4,01/09/2025,09:15:02.975,NIFTY25SEPFUT,13568258,24595.7,75,5700,16610100,24595.7,24605.5,300,300,NIFTY25SEPFUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213543,13/11/2025,15:29:33.077,NIFTY25NOVFUT,9485826,25957.3,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213544,13/11/2025,15:29:34.659,NIFTY25NOVFUT,9485826,25957.5,75,5163450,17647575,25951.3,25957.3,75,150,NIFTY25NOVFUT
1213545,13/11/2025,15:29:35.142,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT
1213546,13/11/2025,15:29:35.834,NIFTY25NOVFUT,9485826,25950.0,75,5164350,17647575,25950.0,25951.0,1725,825,NIFTY25NOVFUT


In [15]:
df.to_parquet("master_fut_df.parquet")

